# Install/Load Lib
SQL stepik https://stepik.org/course/63054


In [1]:
import pandas as pd
import numpy as np

In [2]:
%%capture

# ------------------------------------------------------
# pySpark Tutorial from https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be

!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pickle5
!pip install pyspark_dist_explore

# ------------------------------------------------------
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 4 --executor-memory 4g --driver-memory 3g pyspark-shell'

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType

# ------------------------------------------------------
spark = (
      SparkSession.builder\
        .master("local")\
        .appName("sdr")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.maxResultSize", "4g")\
        #.config("spark.sql.crossJoin.enabled", "true")\
        .config("spark.sql.catalogImplementation","hive")
        .config("spark.sql.hive.convertMetastoreParquet","false")
        .getOrCreate()
      )

import pyspark.sql.functions as F
import pyspark.sql.types as T

# ------------------------------------------------------
spark

In [3]:
# Так как часто предстоит создавать SQL Table из Текста, удобно завести Функцию:

def pDF_to_sDF(
      df_input,
      table_name = 'table1',
      mySchema = None,
      date_columns = [],
      int_columns = [],
      float_columns = [],
      verbose=False
  ):

  df_pandas = df_input.copy()

  # ID -> INT
  for col in [x for x in df_pandas.columns if '_id' in x]:
    try:
      df_pandas[col] = df_pandas[col].astype('float64').astype('int64')
    except:
      df_pd2[col] = pd.to_numeric( df_pd2[col], downcast='integer', errors='coerce' )    
  # INT VAL -> INT
  for col in int_columns:
    try:
      df_pandas[col] = df_pandas[col].astype('float64').astype('int64')
    except:
      df_pd2[col] = pd.to_numeric( df_pd2[col], downcast='integer', errors='coerce' )  
  # FLOAT VAL -> FLOAT
  for col in float_columns:
    try:
      df_pandas[col] = df_pandas[col].astype('float64')        
    except:
      df_pd2[col] = pd.to_numeric( df_pd2[col], downcast='integer', errors='coerce' )  
  # DATE VAL -> DATE
  for col in date_columns:
    df_pandas[col] = pd.to_datetime( df_pandas[col].map(lambda x: x.strip() )
                                      ,format='%Y-%m-%d'
                                      , errors='coerce')

  # Spark Create
  df_sp = spark.createDataFrame(df_pandas, schema=mySchema ) 
  # Time -> Date
  for col in date_columns:
    df_sp = df_sp \
          .withColumn(col, F.to_date(F.col(col)) ) 
  # Int -> Round
  for col in int_columns:
    df_sp = df_sp \
          .withColumn(col, F.round(F.col(col)) ) 

  df_sp.createOrReplaceTempView(table_name)
  # Print()
  if verbose:
    df_sp.printSchema()
    df_sp.show()

  return df_sp, df_pandas

In [4]:
from pygments import highlight
from pygments.lexers import SqlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display

def sprint(sql_query="""--example code formatted
select * from table1;"""):
  display(
      HTML(
          highlight(
              sql_query
              , SqlLexer()
              , HtmlFormatter(full=True)
          )
      )
  )

sprint()

# Data Load/Send to Spark: SQL Table

## 1.6 Таблица "Командировки", запросы на выборку

In [5]:
t = """('Баранов П.Е.','Москва',700 , '2020-01-12', '2020-01-17'),
('Абрамова К.А.','Владивосток',450 , '2020-01-14', '2020-01-27'),
('Семенов И.В.','Москва',700 , '2020-01-23', '2020-01-31'),
('Ильиных Г.Р.','Владивосток', 450, '2020-01-12', '2020-02-02'),
('Колесов С.П.','Москва',700 , '2020-02-01', '2020-02-06'),
('Баранов П.Е.','Москва', 700, '2020-02-14', '2020-02-22'),
('Абрамова К.А.','Москва', 700, '2020-02-23', '2020-03-01'),
('Лебедев Т.К.','Москва', 700, '2020-03-03', '2020-03-06'),
('Колесов С.П.','Новосибирск',450 , '2020-02-27', '2020-03-12'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-03-29', '2020-04-05'),
('Абрамова К.А.','Москва',700 , '2020-04-06', '2020-04-14'),
('Баранов П.Е.','Новосибирск',450 , '2020-04-18', '2020-05-04'),
('Лебедев Т.К.','Томск',450 , '2020-05-20', '2020-05-31'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-06-01', '2020-06-03'),
('Абрамова К.А.','Санкт-Петербург', 700, '2020-05-28', '2020-06-04'),
('Федорова А.Ю.','Новосибирск',450 , '2020-05-25', '2020-06-04'),
('Колесов С.П.','Новосибирск', 450, '2020-06-03', '2020-06-12'),
('Федорова А.Ю.','Томск', 450, '2020-06-20', '2020-06-26'),
('Абрамова К.А.','Владивосток', 450, '2020-07-02', '2020-07-13'),
('Баранов П.Е.','Воронеж', 450, '2020-07-19', '2020-07-25')"""

df_pd = pd.DataFrame( [ x \
                .replace(')','') \
                .replace('(','') \
                .replace("'",'') \
                .split(',') for x in t.split('\n')] ).drop(5,axis=1)
df_pd.columns = ['name','city','per_diem','date_first','date_last']

mySchema = StructType([
                       T.StructField("name", T.StringType(), True)\
                       ,T.StructField("city", T.StringType(), True)\
                       ,T.StructField("per_diem", T.IntegerType(), True)\
                       ,T.StructField("date_first", T.TimestampType(), True)\
                       ,T.StructField("date_last", T.TimestampType(), True)
                       ])

df_sp, df_pd = pDF_to_sDF( df_pd
                          , table_name='trip'
                          , mySchema=mySchema
                          , date_columns=['date_first','date_last']
                          , int_columns=['per_diem']
                          )

df_pd

,name,city,per_diem,date_first,date_last
0,Баранов П.Е.,Москва,700,2020-01-12,2020-01-17
1,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27
2,Семенов И.В.,Москва,700,2020-01-23,2020-01-31
3,Ильиных Г.Р.,Владивосток,450,2020-01-12,2020-02-02
4,Колесов С.П.,Москва,700,2020-02-01,2020-02-06
5,Баранов П.Е.,Москва,700,2020-02-14,2020-02-22
6,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
7,Лебедев Т.К.,Москва,700,2020-03-03,2020-03-06
8,Колесов С.П.,Новосибирск,450,2020-02-27,2020-03-12
9,Семенов И.В.,Санкт-Петербург,700,2020-03-29,2020-04-05


## Практика

###Задание_1
Вывести из таблицы trip информацию о командировках тех сотрудников, фамилия которых заканчивается на букву «а», в отсортированном по убыванию даты последнего дня командировки виде. В результат включить столбцы name, city, per_diem, date_first, date_last.

In [6]:
sql_161 = """SELECT
    name
    , city
    , per_diem
    , date_first
    , date_last

FROM trip

WHERE name LIKE '%а %'
ORDER BY date_last DESC"""

spark.sql(sql_161).toPandas()

,name,city,per_diem,date_first,date_last
0,Абрамова К.А.,Владивосток,450,2020-07-02,2020-07-13
1,Федорова А.Ю.,Томск,450,2020-06-20,2020-06-26
2,Абрамова К.А.,Санкт-Петербург,700,2020-05-28,2020-06-04
3,Федорова А.Ю.,Новосибирск,450,2020-05-25,2020-06-04
4,Абрамова К.А.,Москва,700,2020-04-06,2020-04-14
5,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
6,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27


###Задание_2
Вывести в алфавитном порядке фамилии и инициалы тех сотрудников, которые были в командировке в Москве.

In [7]:
sql_162 = """SELECT
    name

FROM trip

WHERE city = 'Москва'
ORDER BY name ASC"""

spark.sql(sql_162).toPandas()

,name
0,Абрамова К.А.
1,Абрамова К.А.
2,Баранов П.Е.
3,Баранов П.Е.
4,Колесов С.П.
5,Лебедев Т.К.
6,Семенов И.В.


###Задание_3
Для каждого города посчитать, сколько раз сотрудники в нем были.  Информацию вывести в отсортированном в алфавитном порядке по названию городов. Вычисляемый столбец назвать Количество. 

In [8]:
sql_163 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY city ASC"""

spark.sql(sql_163).toPandas()

,city,Kolichestvo
0,Владивосток,3
1,Воронеж,1
2,Москва,7
3,Новосибирск,4
4,Санкт-Петербург,3
5,Томск,2


###Задание_4
Вывести два города, в которых чаще всего были в командировках сотрудники. Вычисляемый столбец назвать Количество.

In [9]:
sql_164 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY Kolichestvo DESC
LIMIT 2"""

spark.sql(sql_164).toPandas()

,city,Kolichestvo
0,Москва,7
1,Новосибирск,4


###Задание_5
Вывести информацию о командировках во все города кроме Москвы и Санкт-Петербурга (фамилии и инициалы сотрудников, город ,  длительность командировки в днях, при этом первый и последний день относится к периоду командировки). Последний столбец назвать Длительность. Информацию вывести в упорядоченном по убыванию длительности поездки, а потом по убыванию названий городов (в обратном алфавитном порядке).

In [10]:
sql_165 = """SELECT
    name
    , city
    , DATEDIFF(date_last,date_first)+1 AS Dlitelnost

FROM trip
WHERE city NOT IN ('Москва','Санкт-Петербург')
ORDER BY Dlitelnost DESC, city DESC"""

spark.sql(sql_165).toPandas()

,name,city,Dlitelnost
0,Ильиных Г.Р.,Владивосток,22
1,Баранов П.Е.,Новосибирск,17
2,Колесов С.П.,Новосибирск,15
3,Абрамова К.А.,Владивосток,14
4,Лебедев Т.К.,Томск,12
5,Абрамова К.А.,Владивосток,12
6,Федорова А.Ю.,Новосибирск,11
7,Колесов С.П.,Новосибирск,10
8,Федорова А.Ю.,Томск,7
9,Баранов П.Е.,Воронеж,7


###Задание_6
Вывести информацию о командировках сотрудника(ов), которые были самыми короткими по времени. В результат включить столбцы name, city, date_first, date_last.

In [11]:
sql_166 = """SELECT

    name, city, date_first, date_last

FROM trip

WHERE (DATEDIFF(date_last,date_first)+1)
            =(
                SELECT MIN(DATEDIFF(date_last,date_first))+1 FROM trip
                )"""

spark.sql(sql_166).toPandas()

,name,city,date_first,date_last
0,Семенов И.В.,Санкт-Петербург,2020-06-01,2020-06-03


###Задание_7
Вывести информацию о командировках, начало и конец которых относятся к одному месяцу (год может быть любой). В результат включить столбцы name, city, date_first, date_last. Строки отсортировать сначала  в алфавитном порядке по названию города, а затем по фамилии сотрудника .

In [12]:
sql_167 = """

SELECT

    name, city, date_first, date_last

FROM trip

WHERE MONTH(date_last)=MONTH(date_first)

ORDER BY city, name

"""

spark.sql(sql_167).toPandas()

,name,city,date_first,date_last
0,Абрамова К.А.,Владивосток,2020-01-14,2020-01-27
1,Абрамова К.А.,Владивосток,2020-07-02,2020-07-13
2,Баранов П.Е.,Воронеж,2020-07-19,2020-07-25
3,Абрамова К.А.,Москва,2020-04-06,2020-04-14
4,Баранов П.Е.,Москва,2020-01-12,2020-01-17
5,Баранов П.Е.,Москва,2020-02-14,2020-02-22
6,Колесов С.П.,Москва,2020-02-01,2020-02-06
7,Лебедев Т.К.,Москва,2020-03-03,2020-03-06
8,Семенов И.В.,Москва,2020-01-23,2020-01-31
9,Колесов С.П.,Новосибирск,2020-06-03,2020-06-12


###Задание_8
Вывести название месяца и количество командировок для каждого месяца. Считаем, что командировка относится к некоторому месяцу, если она началась в этом месяце. Информацию вывести сначала в отсортированном по убыванию количества, а потом в алфавитном порядке по названию месяца виде. Название столбцов – Месяц и Количество.

In [13]:
sql_168 = """

SELECT

    date_format(date_first,'MMMM') AS Month
    , COUNT(*) AS Cnt

FROM trip

GROUP BY date_format(date_first,'MMMM') 

ORDER BY Cnt DESC, Month ASC
;"""

spark.sql(sql_168).toPandas()

,Month,Cnt
0,February,4
1,January,4
2,June,3
3,May,3
4,April,2
5,July,2
6,March,2


###Задание_9
Вывести сумму суточных (произведение количества дней командировки и размера суточных) для командировок, первый день которых пришелся на февраль или март 2020 года. Значение суточных для каждой командировки занесено в столбец per_diem. Вывести фамилию и инициалы сотрудника, город, первый день командировки и сумму суточных. Последний столбец назвать Сумма. Информацию отсортировать сначала  в алфавитном порядке по фамилиям сотрудников, а затем по убыванию суммы суточных.

In [14]:
sql_169 = """

SELECT

    name
    , city
    , date_first
    , (DATEDIFF(date_last,date_first)*per_diem+per_diem) AS Summa

FROM trip

WHERE MONTH(date_first) IN (2,3)

ORDER BY name ASC, Summa DESC;"""

spark.sql(sql_169).toPandas()

,name,city,date_first,Summa
0,Абрамова К.А.,Москва,2020-02-23,5600
1,Баранов П.Е.,Москва,2020-02-14,6300
2,Колесов С.П.,Новосибирск,2020-02-27,6750
3,Колесов С.П.,Москва,2020-02-01,4200
4,Лебедев Т.К.,Москва,2020-03-03,2800
5,Семенов И.В.,Санкт-Петербург,2020-03-29,5600


###Задание_10
Вывести фамилию с инициалами и общую сумму суточных, полученных за все командировки для тех сотрудников, которые были в командировках больше чем 3 раза, в отсортированном по убыванию сумм суточных виде. Последний столбец назвать Сумма.

Только для этого задания изменена строка таблицы trip:

*4	Ильиных Г.Р.	Владивосток	450	2020-01-12	2020-03-02*




In [15]:
sql_1690 = """

SELECT

    name
    , SUM( (DATEDIFF(date_last,date_first)*per_diem+per_diem)  ) AS Summa

FROM trip

WHERE name IN (SELECT name FROM trip GROUP BY name HAVING COUNT(*)>3 )

GROUP BY name

ORDER BY Summa DESC;"""

spark.sql(sql_1690).toPandas()

,name,Summa
0,Абрамова К.А.,29200
1,Баранов П.Е.,21300


## 1.7 Таблица "Нарушения ПДД", запросы корректировки

In [16]:
t1="""('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости(от 40 до 60)', 500.00, '2020-01-12', '2020-01-17'),
       ('Абрамова К.А.', 'О111АВ', 'Проезд на запрещающий сигнал', 1000.00, '2020-01-14', '2020-02-27'),
       ('Яковлев Г.Р.', 'Т330ТТ', 'Превышение скорости(от 20 до 40)', 500.00, '2020-01-23', '2020-02-23'),
       ('Яковлев Г.Р.', 'М701АА', 'Превышение скорости(от 20 до 40)', NULL, '2020-01-12', NULL),
       ('Колесов С.П.', 'К892АХ', 'Превышение скорости(от 20 до 40)', NULL, '2020-02-01', NULL),
       ('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости(от 40 до 60)', NULL, '2020-02-14 ', NULL),
       ('Абрамова К.А.', 'О111АВ', 'Проезд на запрещающий сигнал', NULL, '2020-02-23', NULL),
       ('Яковлев Г.Р.', 'Т330ТТ', 'Проезд на запрещающий сигнал', NULL, '2020-03-03', NULL)"""

df_pd2 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .replace("'",'') \
                .split(',')] for x in t1.split('\n')] ).drop(6,axis=1)

df_pd2.columns = ['name','number_plate','violation','sum_fine','date_violation','date_payment']
df_pd2['name'] = df_pd2['name'].map( lambda x: str(x).replace(')','').replace('(','') )
df_pd2['date_payment'] = df_pd2['date_payment'].map( lambda x: str(x).replace(')','').replace('(','') )

mySchema = StructType([
                       T.StructField("name", T.StringType(), True)\
                       ,T.StructField("number_plate", T.StringType(), True)\
                       ,T.StructField("violation", T.StringType(), True)\
                       ,T.StructField("sum_fine", T.StringType(), True)\

                       ,T.StructField("date_violation", T.TimestampType(), True)\
                       ,T.StructField("date_payment", T.TimestampType(), True)
                       ])

df_sp2, df_pd2 = pDF_to_sDF( df_pd2
                          , table_name='fine'
                          , mySchema=mySchema
                          , date_columns=['date_violation','date_payment']
                          , float_columns=['sum_fine']
                          )

df_pd2

,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500.00,2020-01-12,2020-01-17
1,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.00,2020-01-14,2020-02-27
2,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500.00,2020-01-23,2020-02-23
3,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),NULL,2020-01-12,NaT
4,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NULL,2020-02-01,NaT
5,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NULL,2020-02-14,NaT
6,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NULL,2020-02-23,NaT
7,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NULL,2020-03-03,NaT


In [17]:
t2="""('Превышение скорости(от 20 до 40)', 500),
       ('Превышение скорости(от 40 до 60)', 1000),
       ('Проезд на запрещающий сигнал', 1000)"""

df_pd3 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .replace("'",'') \
                .split(', ')] for x in t2.split('\n')] )#.drop(6,axis=1)

df_pd3.columns = ['violation','sum_fine']

df_pd3['violation'] = df_pd3['violation'].map( lambda x: str(x)[1:] )
df_pd3['sum_fine'] = df_pd3['sum_fine'].map( lambda x: str(x).replace(')','').replace('(','').replace(',','') )

mySchema = StructType([
                       T.StructField("violation", T.StringType(), True)\
                       ,T.StructField("sum_fine", T.IntegerType(), True)
                       ])

df_sp3, df_pd3 = pDF_to_sDF( df_pd3
                          , table_name='traffic_violation'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['sum_fine']
                          )

df_pd3

,violation,sum_fine
0,Превышение скорости(от 20 до 40),500
1,Превышение скорости(от 40 до 60),1000
2,Проезд на запрещающий сигнал,1000


## Практика

### Задание_1
Занести в таблицу fine суммы штрафов, которые должен оплатить водитель, в соответствии с данными из таблицы traffic_violation. При этом суммы заносить только в пустые поля столбца  sum_fine.

Таблица traffic_violationсоздана и заполнена.

In [18]:
sql_170_SQL = """UPDATE fine f, traffic_violation tv
SET f.sum_fine = tv.sum_fine
WHERE tv.violation = f.violation and f.sum_fine IS NULL
;
"""
sprint(sql_170_SQL)



sql_170 = """
select

  f.name
  , f.number_plate
  , f.violation
  , IF(f.sum_fine = 'NaN', tv.sum_fine, f.sum_fine) AS sum_fine
  , f.date_violation
  , f.date_payment

from fine AS f

join traffic_violation AS tv
on tv.violation = f.violation
;

"""

# Таблица fine изменилась

df_sp4 = spark.sql(sql_170)
df_sp4.createOrReplaceTempView("fine")

# df_sp4.printSchema()
# df_sp4.show()

spark.sql(sql_170).toPandas()

,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500.00,2020-01-12,2020-01-17
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NULL,2020-02-14,None
2,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500.00,2020-01-23,2020-02-23
3,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),NULL,2020-01-12,None
4,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NULL,2020-02-01,None
5,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.00,2020-01-14,2020-02-27
6,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NULL,2020-02-23,None
7,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NULL,2020-03-03,None


### Задание_2
Вывести фамилию, номер машины и нарушение только для тех водителей, которые на одной машине нарушили одно и то же правило   два и более раз. При этом учитывать все нарушения, независимо от того оплачены они или нет. Информацию отсортировать в алфавитном порядке, сначала по фамилии водителя, потом по номеру машины и, наконец, по нарушению.

In [19]:
sql_171 = """SELECT

    name
    , number_plate
    , violation

FROM fine

GROUP BY 1,2,3

HAVING COUNT(*)>1

ORDER BY 1,2,3

"""

spark.sql(sql_171).toPandas()

,name,number_plate,violation
0,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60)


###Задание_3
В таблице fine увеличить в два раза сумму неоплаченных штрафов для отобранных на предыдущем шаге записей. 

In [20]:
sql_172_SQL = """UPDATE fine f, ( SELECT
                    name
                    , number_plate
                    , violation
                FROM fine
                GROUP BY 1,2,3
                HAVING COUNT(*)>1 ) nn

SET f.sum_fine = f.sum_fine*2

WHERE f.name = nn.name
and f.number_plate = nn.number_plate
and f.violation = nn.violation
and f.date_payment IS NULL
;"""

sprint(sql_172_SQL)



sql_172 = """
select

  f.name
  , f.number_plate
  , f.violation
  , cast(
         IF( nn.name IS NULL
         , f.sum_fine
         , f.sum_fine*2)
     AS INTEGER) AS sum_fine
  , f.date_violation
  , f.date_payment

from fine AS f

left join ( SELECT
                    name
                    , number_plate
                    , violation
                FROM fine
                GROUP BY 1,2,3
                HAVING COUNT(*)>1 ) nn
on f.name = nn.name
and f.number_plate = nn.number_plate
and f.violation = nn.violation
and f.date_payment is null
;"""

spark.sql(sql_172).toPandas()

,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500.0,2020-01-12,2020-01-17
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NaN,2020-02-14,None
2,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500.0,2020-01-23,2020-02-23
3,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),NaN,2020-01-12,None
4,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NaN,2020-03-03,None
5,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.0,2020-01-14,2020-02-27
6,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NaN,2020-02-23,None
7,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NaN,2020-02-01,None


###Задание_4
Водители оплачивают свои штрафы. В таблице payment занесены даты их оплаты:

<table border="1" cellpadding="0" cellspacing="0">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td><strong>payment_id</strong></td>
			<td><strong>name</strong></td>
			<td><strong>number_plate</strong></td>
			<td><strong>violation</strong></td>
			<td><strong>date_violation</strong></td>
			<td><strong>date_payment</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Яковлев Г.Р.</td>
			<td>М701АА</td>
			<td>Превышение скорости<br>
			(от 20 до 40)</td>
			<td>2020-01-12</td>
			<td>2020-01-22</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Баранов П.Е.</td>
			<td>Р523ВТ</td>
			<td>Превышение скорости<br>
			(от 40 до 60)</td>
			<td>2020-02-14</td>
			<td>2020-03-06</td>
		</tr>
		<tr>
			<td>3</td>
			<td>Яковлев Г.Р.</td>
			<td>Т330ТТ</td>
			<td>Проезд на<br>
			запрещающий сигнал</td>
			<td>2020-03-03</td>
			<td>2020-03-23</td>
		</tr>
	</tbody>
</table>

Необходимо:

в таблицу fine занести дату оплаты соответствующего штрафа из 

*   таблицы payment; 
*   уменьшить начисленный штраф в таблице fine в два раза  (только для тех штрафов, информация о которых занесена в таблицу payment) , если оплата произведена не позднее 20 дней со дня нарушения.

In [21]:
#Загрузим эти данные в Spark SQL

t5 = """('Яковлев Г.Р.', 'М701АА', 'Превышение скорости(от 20 до 40)', '2020-01-12', '2020-01-22'),
('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости(от 40 до 60)', '2020-02-14', '2020-03-06'),
('Яковлев Г.Р.', 'Т330ТТ', 'Проезд на запрещающий сигнал', '2020-03-03', '2020-03-23')"""

df_pd5 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .replace("'",'') \
                .split(', ')] for x in t5.split('\n')] )#.drop(6,axis=1)

df_pd5.columns = ['name', 'number_plate', 'violation', 'date_violation', 'date_payment']

df_pd5['name'] = df_pd5['name'].map( lambda x: str(x).replace(')','').replace('(','') )


mySchema = StructType([
                       T.StructField("name", T.StringType(), True)\
                       ,T.StructField("number_plate", T.StringType(), True)\
                       ,T.StructField("violation", T.StringType(), True)
                       ,T.StructField("date_violation", T.TimestampType(), True)\
                       ,T.StructField("date_payment", T.TimestampType(), True)
                       ])
df_sp5, df_pd5 = pDF_to_sDF( df_pd5
                          , table_name='payment'
                          , mySchema=mySchema
                          , date_columns=['date_violation','date_payment']
                          #, int_columns=['sum_fine']
                          )

df_pd5

,name,number_plate,violation,date_violation,date_payment
0,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),2020-01-12,NaT
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),2020-02-14,NaT
2,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,2020-03-03,NaT


In [22]:
sql_173_SQL = """UPDATE fine f, payment p

SET f.date_payment = p.date_payment
    , f.sum_fine = IF(DATEDIFF(p.date_payment,p.date_violation)<=20,f.sum_fine/2, f.sum_fine)

WHERE f.name = p.name
and f.number_plate = p.number_plate
and f.violation = p.violation
and f.date_violation = p.date_violation
and f.date_payment IS NULL
;"""
sprint(sql_172_SQL)



sql_173 = """
select

  f.name
  , f.number_plate
  , f.violation
  , cast(
         IF( DATEDIFF(p.date_payment,p.date_violation)<=20
            , f.sum_fine/2
            , f.sum_fine )
     AS INTEGER) AS sum_fine
  , f.date_violation
  , IF( p.date_payment IS NULL
        , f.date_payment
        , p.date_payment 
      ) AS date_payment

from fine AS f

left join payment AS p
on f.name = p.name
and f.number_plate = p.number_plate
and f.violation = p.violation
and f.date_violation = p.date_violation
and f.date_payment IS NULL
;"""

# Таблица fine изменилась

df_sp6 = spark.sql(sql_173)
df_sp6.createOrReplaceTempView("fine")

# df_sp6.printSchema()
# df_sp6.show()

spark.sql(sql_173).toPandas()

,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NaN,2020-02-14,None
1,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.0,2020-01-14,2020-02-27
2,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),NaN,2020-01-12,None
3,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NaN,2020-02-23,None
4,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500.0,2020-01-12,2020-01-17
5,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NaN,2020-02-01,None
6,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NaN,2020-03-03,None
7,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500.0,2020-01-23,2020-02-23


###Задание_5
Создать новую таблицу back_payment, куда внести информацию о неоплаченных штрафах (Фамилию и инициалы водителя, номер машины, нарушение, сумму штрафа  и  дату нарушения) из таблицы fine.

In [23]:
sql_174_SQL = """CREATE TABLE back_payment
(
  SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation

    FROM fine AS f

    WHERE f.date_payment IS NULL      
);

SELECT * FROM back_payment;
"""
sprint(sql_174_SQL)



sql_174 = """SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation

    FROM fine AS f

    WHERE f.date_payment IS NULL
    ;"""

df_sp7 = spark.sql(sql_174)
df_sp7.createOrReplaceTempView("back_payment")

spark.sql('select * from back_payment').toPandas()

,name,number_plate,violation,sum_fine,date_violation
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NaN,2020-02-14
1,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),NaN,2020-01-12
2,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NaN,2020-02-23
3,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NaN,2020-02-01
4,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NaN,2020-03-03


###Задание_6
Удалить из таблицы fine информацию о нарушениях, совершенных раньше 1 февраля 2020 года. 

In [24]:
sql_175_SQL = """DELETE FROM fine 
WHERE date_violation < DATE'2020-02-01'
;
    
SELECT * FROM fine;"""
sprint(sql_175_SQL)



sql_175 = """
SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation
      , f.date_payment

    FROM fine AS f
    
    WHERE f.date_violation >= DATE'2020-02-01'
"""

df_sp7 = spark.sql(sql_175)
df_sp7.createOrReplaceTempView("fine")

spark.sql('select * from fine').toPandas()

,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NaN,2020-02-14,None
1,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NaN,2020-02-23,None
2,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NaN,2020-02-01,None
3,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NaN,2020-03-03,None


### Примечание

<span><h2>Поиск по ключевым словам</h2>

<p>На данном шаге можно найти шаги курса, в которых встречаются ключевые слова SQL, которые рассматриваются в курсе.</p>

<p>Для этого скопируйте&nbsp;один из запросов&nbsp;в окно решений, укажите нужные ключевые слова и запустите запрос. В окне решений будут выведены ссылки на соответствующие шаги.</p>

<p>Это <strong>НЕ ЗАДАНИЕ</strong>, а просто запросы, с помощью которых можно найти шаги, в которых встречаются те или иные ключевые слова. Выполнять не обязательно (это задание оценивается в 0 баллов). Это ПРОСТО ПОМОЩЬ для навигации по курсу.</p>

<p><strong>Запрос 1.&nbsp;</strong>Поиск шагов,&nbsp;в которых встречается заданное ключевое слово, в примере <strong>MAX</strong>:</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> 
   <span class="hljs-keyword">CONCAT</span>(module_id,<span class="hljs-string">'.'</span>,lesson_position,<span class="hljs-string">"."</span>,step_position,<span class="hljs-string">" "</span>, <span class="hljs-keyword">CONCAT</span>(<span class="hljs-keyword">LEFT</span>(step_name, <span class="hljs-number">50</span>), <span class="hljs-string">'...'</span>)) <span class="hljs-keyword">AS</span> Шаг,
   note <span class="hljs-keyword">AS</span> Примечание
<span class="hljs-keyword">FROM</span> step
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> lesson <span class="hljs-keyword">USING</span>(lesson_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> <span class="hljs-keyword">module</span> <span class="hljs-keyword">USING</span>(module_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> step_keyword <span class="hljs-keyword">USING</span>(step_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> keyword <span class="hljs-keyword">USING</span>(keyword_id)
<span class="hljs-keyword">WHERE</span> keyword_name = <span class="hljs-string">'MAX'</span>
<span class="hljs-keyword">ORDER</span> <span class="hljs-keyword">BY</span> <span class="hljs-number">1</span>;</code></pre>

<p><strong>Запрос 2.&nbsp;</strong>Поиск шагов,&nbsp;в которых встречаются два заданных ключевых слова одновременно, в примере <strong>MAX</strong> и <strong>AVG</strong>:</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> 
   <span class="hljs-keyword">CONCAT</span>(module_id,<span class="hljs-string">'.'</span>,lesson_position,<span class="hljs-string">"."</span>,step_position,<span class="hljs-string">" "</span>, <span class="hljs-keyword">CONCAT</span>(<span class="hljs-keyword">LEFT</span>(step_name, <span class="hljs-number">30</span>), <span class="hljs-string">'...'</span>)) <span class="hljs-keyword">AS</span> Шаг, 
   <span class="hljs-keyword">link</span> <span class="hljs-keyword">AS</span> Ссылка_на_шаг
<span class="hljs-keyword">FROM</span> step
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> lesson <span class="hljs-keyword">USING</span>(lesson_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> <span class="hljs-keyword">module</span> <span class="hljs-keyword">USING</span>(module_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> step_keyword <span class="hljs-keyword">USING</span>(step_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> keyword <span class="hljs-keyword">USING</span>(keyword_id)
<span class="hljs-keyword">WHERE</span> keyword_name <span class="hljs-keyword">IN</span> (<span class="hljs-string">'MAX'</span>, <span class="hljs-string">'AVG'</span>)
<span class="hljs-keyword">GROUP</span> <span class="hljs-keyword">BY</span> ШАГ, Ссылка_на_шаг
<span class="hljs-keyword">HAVING</span> <span class="hljs-keyword">count</span>(*) = <span class="hljs-number">2</span>
<span class="hljs-keyword">ORDER</span> <span class="hljs-keyword">BY</span> <span class="hljs-number">1</span>;</code></pre>

<p><strong>&nbsp;Запрос 3.&nbsp;</strong>Поиск шагов,&nbsp;в которых встречаются три заданных ключевых слова одновременно, в примере <strong>MAX</strong>, <strong>MIN</strong> и <strong>AVG</strong>:</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> 
   <span class="hljs-keyword">CONCAT</span>(module_id,<span class="hljs-string">'.'</span>,lesson_position,<span class="hljs-string">"."</span>,step_position,<span class="hljs-string">" "</span>,step_name) <span class="hljs-keyword">AS</span> Шаг
<span class="hljs-keyword">FROM</span> step
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> lesson <span class="hljs-keyword">USING</span>(lesson_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> <span class="hljs-keyword">module</span> <span class="hljs-keyword">USING</span>(module_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> step_keyword <span class="hljs-keyword">USING</span>(step_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> keyword <span class="hljs-keyword">USING</span>(keyword_id)
<span class="hljs-keyword">WHERE</span> keyword_name <span class="hljs-keyword">IN</span> (<span class="hljs-string">'MAX'</span>, <span class="hljs-string">'AVG'</span>, <span class="hljs-string">'MIN'</span>)
<span class="hljs-keyword">GROUP</span> <span class="hljs-keyword">BY</span> ШАГ
<span class="hljs-keyword">HAVING</span> <span class="hljs-keyword">COUNT</span>(*) = <span class="hljs-number">3</span>
<span class="hljs-keyword">ORDER</span> <span class="hljs-keyword">BY</span> <span class="hljs-number">1</span>;</code></pre></span>

## 2.1 Связи между таблицами

## Практика

In [25]:
#Загрузим эти данные в Spark SQL

sql_210 = """INSERT INTO author(name_author)
VALUES
    ('Булгаков М.А.'),
    ('Достоевский Ф.М.'),
    ('Есенин С.А.'),
    ('Пастернак Б.Л.')
;"""
sprint(sql_210)


t6 = """('Булгаков М.А.'),
       ('Достоевский Ф.М.'),
       ('Есенин С.А.'),
       ('Пастернак Б.Л.')"""

df_pd8 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("'")] for x in t6.split('\n')] ).drop([0,2],axis=1)
 
df_pd8 = df_pd8.reset_index()
df_pd8.columns = ['author_id','name_author']
df_pd8['author_id'] = df_pd8['author_id']+1

mySchema = StructType([
                        T.StructField("author_id", T.IntegerType(), True)
                       , T.StructField("name_author", T.StringType(), True)\
                       ])

df_sp8, df_pd8 = pDF_to_sDF( df_pd8
                          , table_name='author'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['sum_fine']
                          )

df_pd8

,author_id,name_author
0,1,Булгаков М.А.
1,2,Достоевский Ф.М.
2,3,Есенин С.А.
3,4,Пастернак Б.Л.


### Задание_1
<p>Перепишите&nbsp;запрос на создание таблицы&nbsp;<code><strong>book</strong></code>&nbsp;, чтобы ее структура соответствовала структуре, показанной на логической схеме (таблица <code><strong>genre</strong></code> уже создана, порядок следования столбцов - как на логической схеме в таблице <code><strong>book</strong></code>, <code><strong>genre_id</strong></code>&nbsp; - внешний ключ)&nbsp;. Для <strong><code>genre_id</code></strong>&nbsp;ограничение о недопустимости пустых значений <strong>не задавать</strong>. В качестве главной таблицы для описания поля&nbsp; <strong><code>genre_id</code></strong>использовать таблицу <a href="https://stepik.org/lesson/297508/step/5?unit=279268" rel="noopener noreferrer nofollow"><code><strong>genre</strong></code></a> следующей структуры:</p>

<table border="1" cellpadding="1" cellspacing="1">
	<tbody>
		<tr style="background-color: #e6f8e0; background: #e6f8e0; text-align: center;">
			<td style="text-align: center;"><strong>Поле</strong></td>
			<td style="text-align: center;"><strong>Тип, описание</strong></td>
		</tr>
		<tr>
			<td><code>genre_id</code></td>
			<td><code>INT PRIMARY KEY AUTO_INCREMENT</code></td>
		</tr>
		<tr>
			<td><code>name_genre</code></td>
			<td><code>VARCHAR(30)</code></td>
		</tr>
	</tbody>
</table>

<p><strong>Логическая схема</strong> (нужно создать только таблицу <strong><code>book</code></strong>):</p>

<p><img alt="" src="https://ucarecdn.com/95045d96-412d-4e10-88f2-7ac6b13fada6/"></p>

In [26]:
sql_211 = """CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    title VARCHAR(50), 
    author_id INT NOT NULL, 
    genre_id INT, 
    
    price DECIMAL(8,2), 
    amount INT, 
    FOREIGN KEY (genre_id)  REFERENCES genre (genre_id) ,
    FOREIGN KEY (author_id)  REFERENCES author (author_id) 
);"""
sprint(sql_211)

### Задание_2
<p>Создать таблицу <code><strong>book</strong></code> той же структуры, что и на предыдущем шаге.&nbsp;Будем считать, что при удалении автора из таблицы <code><strong>author</strong></code>, должны удаляться все записи о книгах из таблицы <code><strong>book</strong></code>, написанные этим автором. А при удалении жанра из таблицы <code><strong>genre</strong></code> для соответствующей записи <code><strong>book</strong></code> установить значение <code><strong>Null </strong></code>в столбце <code><strong>genre_id</strong></code>.&nbsp;</p>

<span><h2>Действия при&nbsp;удалении записи главной таблицы</h2>

<p>С помощью выражения<strong>&nbsp;</strong><code>ON DELETE</code>&nbsp;можно установить действия, которые выполняются для записей подчиненной таблицы при удалении связанной строки из главной таблицы. При удалении можно установить следующие опции:</p>

<ul>
	<li><code>CASCADE</code>: автоматически удаляет строки из зависимой таблицы при удалении&nbsp; связанных строк в главной таблице.</li>
	<li><code>SET NULL</code>: при удалении&nbsp; связанной строки из главной таблицы устанавливает для столбца внешнего ключа значение&nbsp;<strong><code>NULL</code></strong>. (В этом случае столбец внешнего ключа должен поддерживать установку<code><strong> NULL</strong></code>).</li>
	<li><code>SET&nbsp;DEFAULT</code>&nbsp;похоже на <code>SET&nbsp;NULL</code> за тем исключением, что значение&nbsp; внешнего ключа устанавливается не в <code>NULL,</code>&nbsp;а в значение по умолчанию для данного столбца.</li>
	<li><code>RESTRICT</code>: отклоняет удаление строк в главной таблице при наличии связанных строк в зависимой таблице.</li>
</ul>

<p><strong>Важно! </strong>Если для столбца установлена опция <code>SET NULL</code>, то при его описании нельзя задать ограничение на пустое значение.</p>

In [27]:
sql_122 = """CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    title VARCHAR(50), 
    author_id INT NOT NULL, 
    genre_id INT, 
    
    price DECIMAL(8,2), 
    amount INT, 
    FOREIGN KEY (genre_id)  REFERENCES genre (genre_id) ON DELETE SET NULL,
    FOREIGN KEY (author_id)  REFERENCES author (author_id)  ON DELETE CASCADE
);"""
sprint(sql_122)

### Задание_3

<div id="ember15823" class="html-content rich-text-viewer ember-view" data-processed=""><!----><span><h2>Заполнение таблицы с внешними ключами</h2>

<p>На предыдущих шагах были созданы и заполнены таблицы<code><strong> author</strong></code>:</p>

<table border="1" cellpadding="1" cellspacing="1">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td style="text-align: center;"><strong>author_id</strong></td>
			<td style="text-align: center;"><strong>name_author</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Булгаков М.А.</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Достоевский Ф.М.</td>
		</tr>
		<tr>
			<td>3</td>
			<td>Есенин С.А.</td>
		</tr>
		<tr>
			<td>4</td>
			<td>Пастернак Б.Л.</td>
		</tr>
	</tbody>
</table>

<p>и&nbsp;&nbsp;<code><strong>genre</strong></code>:</p>

<table border="1" cellpadding="1" cellspacing="1">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td style="text-align: center;"><strong>genre_id</strong></td>
			<td style="text-align: center;"><strong>name_genre</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Роман</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Поэзия</td>
		</tr>
	</tbody>
</table>


<h2>Задание</h2>

<p>Для каждой строки таблицы <code><strong>book</strong></code> занесите значения в поля <code><strong>author_id </strong></code>и <code><strong>genre_id</strong></code>. Считать, что книга Есенина относится к жанру «Поэзия», остальные книги – к жанру «Роман».</p>

<p>Через запятую перечислены&nbsp;значения полей&nbsp;<code><strong>book_id</strong></code>,&nbsp; <code><strong>title</strong></code>, <code><strong>author_id</strong></code>, <code><strong>genre_id</strong></code>,&nbsp;&nbsp;<code><strong>price</strong></code>, <code><strong>amount</strong></code> каждой записи таблицы <code><strong>book</strong></code>. Заполните пропуски.</p>

<p>Авторы и их произведения:</p>

<table border="1" cellpadding="0" cellspacing="0">
	<tbody>
		<tr style="background-color: #dcdcdc; background: #dcdcdc; text-align: center;">
			<td>
			<strong>Название книги</strong>
			</td>
			<td>
			<strong>Автор</strong>
			</td>
			<td>
			<strong>Цена</strong>
			</td>
			<td>
			<strong>Количество</strong>
			</td>
		</tr>
		<tr>
			<td>Мастер и Маргарита</td>
			<td>Булгаков М.А.</td>
			<td>670.99</td>
			<td>3</td>
		</tr>
		<tr>
			<td>Белая гвардия</td>
			<td>Булгаков М.А.</td>
			<td>540.50</td>
			<td>5</td>
		</tr>
		<tr>
			<td>Идиот</td>
			<td>Достоевский Ф.М.</td>
			<td>460.00</td>
			<td>10</td>
		</tr>
		<tr>
			<td>Братья Карамазовы</td>
			<td>Достоевский Ф.М.</td>
			<td>799.01</td>
			<td>3</td>
		</tr>
		<tr>
			<td>Игрок</td>
			<td>Достоевский Ф.М.</td>
			<td>480.50</td>
			<td>10</td>
		</tr>
		<tr>
			<td>Стихотворения и поэмы</td>
			<td>Есенин С.А.</td>
			<td>650.00</td>
			<td>15</td>
		</tr>
	</tbody>
</table></span></div>

### Задание_4
<p>Добавьте три последние записи (с ключевыми значениями 6, 7, 8) в таблицу<code><strong> book</strong></code>, первые 5 записей уже добавлены:</p>

<table border="1" cellpadding="0" cellspacing="0">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td><strong>book_id</strong></td>
			<td><strong>title</strong></td>
			<td><strong>author_id</strong></td>
			<td><strong>genre_id</strong></td>
			<td><strong>price</strong></td>
			<td><strong>amount</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Мастер и Маргарита</td>
			<td>1</td>
			<td>1</td>
			<td>670.99</td>
			<td>3</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Белая гвардия</td>
			<td>1</td>
			<td>1</td>
			<td>540.50</td>
			<td>5</td>
		</tr>
		<tr>
			<td>3</td>
			<td>Идиот</td>
			<td>2</td>
			<td>1</td>
			<td>460.00</td>
			<td>10</td>
		</tr>
		<tr>
			<td>4</td>
			<td>Братья Карамазовы</td>
			<td>2</td>
			<td>1</td>
			<td>799.01</td>
			<td>3</td>
		</tr>
		<tr>
			<td>5</td>
			<td>Игрок</td>
			<td>2</td>
			<td>1</td>
			<td>480.50</td>
			<td>10</td>
		</tr>
		<tr>
			<td>6</td>
			<td>Стихотворения и поэмы</td>
			<td>3</td>
			<td>2</td>
			<td>650.00</td>
			<td>15</td>
		</tr>
		<tr>
			<td>7</td>
			<td>Черный человек</td>
			<td>3</td>
			<td>2</td>
			<td>570.20</td>
			<td>6</td>
		</tr>
		<tr>
			<td>8</td>
			<td>Лирика</td>
			<td>4</td>
			<td>2</td>
			<td>518.99</td>
			<td>2</td>
		</tr>
	</tbody>
</table>

<p><strong>Логическая схема базы данных:</strong></p>

<p><img alt="" src="https://ucarecdn.com/26d5d90a-3e95-46bc-807d-10fb53d10f25/"></p>

<details><summary><strong>Результат</strong></summary>

<pre><code class="language-sql hljs">Affected rows: 1
Affected rows: 1
Affected rows: 1
Query result:
+<span class="hljs-comment">---------+-----------------------+-----------+----------+--------+--------+</span>
| book_id | title                 | author_id | genre_id | price  | amount |
+<span class="hljs-comment">---------+-----------------------+-----------+----------+--------+--------+</span>
| 1       | Мастер и Маргарита    | 1         | 1        | 670.99 | 3      |
| 2       | Белая гвардия         | 1         | 1        | 540.50 | 5      |
| 3       | Идиот                 | 2         | 1        | 460.00 | 10     |
| 4       | Братья Карамазовы     | 2         | 1        | 799.01 | 3      |
| 5       | Игрок                 | 2         | 1        | 480.50 | 10     |
| 6       | Стихотворения и поэмы | 3         | 2        | 650.00 | 15     |
| 7       | Черный человек        | 3         | 2        | 570.20 | 6      |
| 8       | Лирика                | 4         | 2        | 518.99 | 2      |
+<span class="hljs-comment">---------+-----------------------+-----------+----------+--------+--------+</span></code></pre>
</details></span>

In [28]:
sql_123 = """INSERT INTO book(title,author_id,genre_id,price,amount)
VALUES     ('Стихотворения и поэмы',3,2,650.00,15),
        ('Черный человек',3,2,570.20,6),
        ('Лирика',4,2,518.99,2)
;"""
sprint(sql_123)

### Задание_5
Вывести название, жанр и цену тех книг, количество которых больше 8, в отсортированном по убыванию цены виде.

In [29]:
t7 = """1	Мастер и Маргарита	1	1	670.99	3
2	Белая гвардия	1	1	540.50	5
3	Идиот	2	1	460.00	10
4	Братья Карамазовы	2	1	799.01	3
5	Игрок	2	1	480.50	10
6	Стихотворения и поэмы	3	2	650.00	15
7	Черный человек	3	2	570.20	6
8	Лирика	4	2	518.99	2"""

df_pd9 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t7.split('\n')] ).drop([0],axis=1)
 
df_pd9.columns = 'title	author_id	genre_id	price	amount'.split('\t')

mySchema = StructType([ T.StructField("title", T.StringType(), True) \
                       , T.StructField("author_id", T.IntegerType(), True)\
                       , T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("price", T.FloatType(), True) \
                       , T.StructField("amount", T.IntegerType(), True)
                       ])

df_sp9, df_pd9 = pDF_to_sDF( df_pd9
                          , table_name='book'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['amount']
                          , float_columns=['price']
                          )

df_pd9

,title,author_id,genre_id,price,amount
0,Мастер и Маргарита,1,1,670.99,3
1,Белая гвардия,1,1,540.50,5
2,Идиот,2,1,460.00,10
3,Братья Карамазовы,2,1,799.01,3
4,Игрок,2,1,480.50,10
5,Стихотворения и поэмы,3,2,650.00,15
6,Черный человек,3,2,570.20,6
7,Лирика,4,2,518.99,2


In [30]:
t7 = """1	Роман
2	Поэзия"""

df_pd10 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t7.split('\n')] )
 
df_pd10.columns = 'genre_id	name_genre'.split('\t')

mySchema = StructType([ T.StructField("genre_id", T.IntegerType(), True) \
                       , T.StructField("name_genre", T.StringType(), True)
                       ])
df_sp10, df_pd10 = pDF_to_sDF( df_pd10
                          , table_name='genre'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )
df_pd10

,genre_id,name_genre
0,1,Роман
1,2,Поэзия


In [31]:
sql_124 = """SELECT 
    book.title
    , genre.name_genre
    , book.price
FROM author 
INNER JOIN book
ON author.author_id = book.author_id
INNER JOIN genre
ON genre.genre_id = book.genre_id
WHERE book.amount > 8
ORDER BY book.price DESC
;"""

spark.sql(sql_124).toPandas()

,title,name_genre,price
0,Стихотворения и поэмы,Поэзия,650.0
1,Игрок,Роман,480.5
2,Идиот,Роман,460.0


### Задание_6

Вывести все жанры, которые не представлены в книгах на складе.

In [32]:
# Добавить жанр "Приключения"

mySchema = StructType([
                       T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("name_genre", T.StringType(), True)
                       ])


df_sp10, df_pd10 = pDF_to_sDF( df_pd10.append( 
                                      pd.DataFrame( [[3,'Приключения']]
                                                   , columns=df_pd10.columns )
                                  )
                          , table_name='genre'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd10

,genre_id,name_genre
0,1,Роман
1,2,Поэзия
0,3,Приключения


In [33]:
sql_125 = """SELECT 
    genre.name_genre
    
FROM genre

LEFT JOIN book
ON genre.genre_id = book.genre_id

WHERE book.title IS NULL
;"""

spark.sql(sql_125).toPandas()

,name_genre
0,Приключения


### Задание_7
Необходимо в каждом городе провести выставку книг каждого автора в течение 2020 года. Дату проведения выставки выбрать случайным образом. Создать запрос, который выведет город, автора и дату проведения выставки. Последний столбец назвать Дата. Информацию вывести, отсортировав сначала в алфавитном порядке по названиям городов, а потом по убыванию дат проведения выставок.

<details open=""><summary><strong>Пояснение</strong></summary>

<p>1. Для генерации случайной даты можно к&nbsp;первому числу года ('2020-01-01') прибавить целое случайное число в интервале от 0 до 365.</p>

<p>Генерации случайных чисел в интервале от 0 до 1 (не включительно)&nbsp;осуществляется с помощью функции <code>RAND()</code>. Если эту функцию умножить на 365, то она будет генерировать вещественные числа от 0 до 365 (не включительно). Осталось только отбросить дробную часть. Это можно сделать с помощью функции <code>FLOOR()</code>, которая возвращает наибольшее целое число, меньшее или равное указанному числовому значению. Таким образом, случайное число от 0 до 365 можно получить с помощью выражения:</p>

<pre><code class="hljs excel"><span class="hljs-built_in">FLOOR</span>(<span class="hljs-built_in">RAND</span>() * <span class="hljs-number">365</span>)</code></pre>

<p><strong>Важно! </strong>Даты должны быть за 2020 год, первое число года - 1 января 2020 года.</p>

<p>2. Для сложения&nbsp; даты с числом используется функция:</p>

<pre><code class="language-sql hljs">DATE_ADD(дата, INTERVAL число единица_измерения),

где
  единица_измерения (использовать прописные буквы) – это день (DAY), месяц(MONTH), неделя(WEEK) и пр., 
  число – целое число,
  дата – значение даты или даты и времени.</code></pre>

<p>Функция к <strong>дате</strong>&nbsp; прибавляет указанное <strong>число</strong>, выраженное в днях, месяцах и пр. , в зависимости от заданного интервала, и возвращает новую дату.</p>

<p>Например:</p>

<pre><code class="language-sql hljs">DATE_ADD('2020-02-02', INTERVAL 45 DAY) возвращает 18 марта 2020 года
DATE_ADD('2020-02-02', INTERVAL 6 MONTH) возвращает 2 августа 2020 года</code></pre>
</details>

In [34]:
# Создадим таблицу city

t8 = """1	Москва
2	Санкт-Петербург
3	Владивосток"""

df_pd11 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t8.split('\n')] )
 
df_pd11.columns = 'city_id	name_city'.split('\t')

mySchema = StructType([ T.StructField("city_id", T.IntegerType(), True)\
                       , T.StructField("name_city", T.StringType(), True)
                       ])
df_sp11, df_pd11 = pDF_to_sDF( df_pd11
                          , table_name='city'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd11

,city_id,name_city
0,1,Москва
1,2,Санкт-Петербург
2,3,Владивосток


In [35]:
sql_126_SQL = """SELECT

    city.name_city
    , author.name_author
    , DATE_ADD('2020-01-01', INTERVAL FLOOR(RAND() * 365) DAY) AS date_dt

FROM city

CROSS JOIN author
ON 1=1

ORDER BY city.name_city ASC
    , date_dt DESC
;"""
sprint(sql_126_SQL)



sql_126 = """SELECT

    city.name_city
    , author.name_author
    , DATE_ADD( DATE ('2020-01-01'), CAST( FLOOR(RAND()*365) AS INTEGER) ) AS date_dt

FROM city

CROSS JOIN author
ON 1=1

ORDER BY city.name_city ASC
    , date_dt DESC
;"""
spark.sql(sql_126).toPandas()

,name_city,name_author,date_dt
0,Владивосток,Достоевский Ф.М.,2020-10-19
1,Владивосток,Булгаков М.А.,2020-07-31
2,Владивосток,Пастернак Б.Л.,2020-04-18
3,Владивосток,Есенин С.А.,2020-01-22
4,Москва,Булгаков М.А.,2020-11-25
5,Москва,Есенин С.А.,2020-09-26
6,Москва,Достоевский Ф.М.,2020-08-28
7,Москва,Пастернак Б.Л.,2020-07-04
8,Санкт-Петербург,Есенин С.А.,2020-11-28
9,Санкт-Петербург,Пастернак Б.Л.,2020-08-17


### Задание_8
Вывести информацию о книгах (жанр, книга, автор), относящихся к жанру, включающему слово «роман» в отсортированном по названиям книг виде.

In [36]:
sql_127 = """SELECT

    genre.name_genre
    , book.title
    , author.name_author

FROM author

INNER JOIN book
ON author.author_id = book.author_id

INNER JOIN genre
ON genre.genre_id = book.genre_id

WHERE lower(genre.name_genre) LIKE '%роман%'

ORDER BY book.title ASC
;"""

spark.sql(sql_127).toPandas()

,name_genre,title,name_author
0,Роман,Белая гвардия,Булгаков М.А.
1,Роман,Братья Карамазовы,Достоевский Ф.М.
2,Роман,Игрок,Достоевский Ф.М.
3,Роман,Идиот,Достоевский Ф.М.
4,Роман,Мастер и Маргарита,Булгаков М.А.


### Задание_9
<p>Посчитать количество экземпляров&nbsp; книг каждого автора из таблицы <code><strong>author</strong></code>.&nbsp; Вывести тех авторов,&nbsp;&nbsp;количество книг которых меньше 10, в отсортированном по возрастанию количества виде. Последний столбец назвать&nbsp;<code><strong>Количество</strong></code>.</p>

In [37]:
mySchema = StructType([
                        T.StructField("author_id", T.IntegerType(), True)
                       , T.StructField("name_author", T.StringType(), True)\
                       ])
df_sp8, df_pd8 = pDF_to_sDF( df_pd8.append(
                                            pd.DataFrame( [(5,'Лермонтов М.Ю.')]
                                                          , columns=df_pd8.columns)
                                            )
                          , table_name='author'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd8

,author_id,name_author
0,1,Булгаков М.А.
1,2,Достоевский Ф.М.
2,3,Есенин С.А.
3,4,Пастернак Б.Л.
0,5,Лермонтов М.Ю.


In [38]:
sql_128 = """SELECT 
    name_author
    , sum(amount) AS Kolichestvo

FROM author 

LEFT JOIN book
    on author.author_id = book.author_id

GROUP BY name_author

HAVING ( Coalesce(sum(amount),0) < 10 ) 

ORDER BY Kolichestvo ASC
;"""

spark.sql(sql_128).toPandas()

,name_author,Kolichestvo
0,Лермонтов М.Ю.,NaN
1,Пастернак Б.Л.,2.0
2,Булгаков М.А.,8.0


### Задание_10
Вывести в алфавитном порядке всех авторов, которые пишут только в одном жанре. Поскольку у нас в таблицах так занесены данные, что у каждого автора книги только в одном жанре,  для этого запроса внесем изменения в таблицу book. Пусть у нас  книга Есенина «Черный человек» относится к жанру «Роман», а книга Булгакова «Белая гвардия» к «Приключениям» (эти изменения в таблицы уже внесены).



In [39]:
# Создадим таблицу book

t9 = """| book_id | title                 | author_id | genre_id | price  | amount |
| 1       | Мастер и Маргарита    | 1         | 1        | 670.99 | 3      |
| 2       | Белая гвардия         | 1         | 3        | 540.50 | 5      |
| 3       | Идиот                 | 2         | 1        | 460.00 | 10     |
| 4       | Братья Карамазовы     | 2         | 1        | 799.01 | 3      |
| 5       | Игрок                 | 2         | 1        | 480.50 | 10     |
| 6       | Стихотворения и поэмы | 3         | 2        | 650.00 | 15     |
| 7       | Черный человек        | 3         | 1        | 570.20 | 6      |
| 8       | Лирика                | 4         | 2        | 518.99 | 2      |"""

df_pd12 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("|")] for x in t9.split('\n')] ).drop([0,7],axis=1)

df_pd12.columns = df_pd12.iloc[0,:]
df_pd12 = df_pd12.iloc[1:,:]
df_pd12 = df_pd12.drop(['book_id'],axis=1)

mySchema = StructType([
                        T.StructField("title", T.StringType(), True)
                       , T.StructField("author_id", T.IntegerType(), True)\
                       , T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("price", T.FloatType(), True)\
                       , T.StructField("amount", T.IntegerType(), True)\
                       ])
df_sp12, df_pd12 = pDF_to_sDF( df_pd12
                          , table_name='book'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['amount']
                          , float_columns=['price']
                          )

df_pd12

,title,author_id,genre_id,price,amount
1,Мастер и Маргарита,1,1,670.99,3
2,Белая гвардия,1,3,540.50,5
3,Идиот,2,1,460.00,10
4,Братья Карамазовы,2,1,799.01,3
5,Игрок,2,1,480.50,10
6,Стихотворения и поэмы,3,2,650.00,15
7,Черный человек,3,1,570.20,6
8,Лирика,4,2,518.99,2


In [40]:
sql_129 = """SELECT 
    name_author
    --, COUNT( DISTINCT genre_id) AS cnt_genre_id
FROM book 
INNER JOIN author
    on author.author_id = book.author_id 
GROUP BY name_author
HAVING COUNT( DISTINCT genre_id) = 1
ORDER BY name_author
;"""

spark.sql(sql_129).toPandas()

,name_author
0,Достоевский Ф.М.
1,Пастернак Б.Л.


### Задание_11
Вывести информацию о книгах (название книги, фамилию и инициалы автора, название жанра, цену и количество экземпляров книги), написанных в самых популярных жанрах, в отсортированном в алфавитном порядке по названию книг виде. Самым популярным считать жанр, общее количество экземпляров книг которого на складе максимально.

In [41]:
sql_1291 = """SELECT  

    book.title
    , author.name_author
    , genre.name_genre
    , CAST ( book.price AS DECIMAL(18,2) ) AS price
    , book.amount

FROM  author 

INNER JOIN book 
ON author.author_id = book.author_id

INNER JOIN genre 
ON  book.genre_id = genre.genre_id

WHERE genre.genre_id IN
         ( SELECT query_in_1.genre_id
          FROM 
              ( /* выбираем код жанра и количество произведений, относящихся к нему */
                SELECT genre_id, SUM(amount) AS sum_amount
                FROM book
                GROUP BY genre_id
               )query_in_1
          INNER JOIN 
              ( /* выбираем запись, в которой указан код жанр с максимальным количеством книг */
                SELECT genre_id, SUM(amount) AS sum_amount
                FROM book
                GROUP BY genre_id
                ORDER BY sum_amount DESC
                LIMIT 1
               ) query_in_2
          ON query_in_1.sum_amount = query_in_2.sum_amount
         )
ORDER BY book.title;  """

spark.sql(sql_1291).toPandas()

,title,name_author,name_genre,price,amount
0,Братья Карамазовы,Достоевский Ф.М.,Роман,799.01,3
1,Игрок,Достоевский Ф.М.,Роман,480.50,10
2,Идиот,Достоевский Ф.М.,Роман,460.00,10
3,Мастер и Маргарита,Булгаков М.А.,Роман,670.99,3
4,Черный человек,Есенин С.А.,Роман,570.20,6


### Задание_12

<p>Если в таблицах&nbsp;<code><strong>supply</strong></code>&nbsp; и&nbsp;<code><strong>book</strong></code> есть одинаковые книги, которые имеют равную цену,&nbsp; вывести их название и автора, а также посчитать общее количество экземпляров книг в таблицах&nbsp;<code><strong>supply</strong></code>&nbsp;и&nbsp;<code><strong>book</strong></code>,&nbsp; столбцы назвать <strong><code>Название</code>, <code>Автор</code></strong>&nbsp; и&nbsp;<strong><code>Количество</code>.</strong></p>

In [42]:
# Создадим таблицу supply

t10 = """supply_id	title	author	price	amount
1	Доктор Живаго	Пастернак Б.Л.	618.99	3
2	Черный человек 	Есенин С.А.	570.20	6
3	Евгений Онегин	Пушкин А.С.	440.80	5
4	Идиот	Достоевский Ф.М.	360.80	3"""

df_pd13 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t10.split('\n')] )#.drop([0,7],axis=1)
 
df_pd13.columns = df_pd13.iloc[0,:]
df_pd13 = df_pd13.iloc[1:,:]

mySchema = StructType([ T.StructField("supply_id", T.IntegerType(), True)\
                       , T.StructField("title", T.StringType(), True)
                       , T.StructField("author", T.StringType(), True)\
                       , T.StructField("price", T.FloatType(), True)\
                       , T.StructField("amount", T.IntegerType(), True)\
                       ])
df_sp13, df_pd13 = pDF_to_sDF( df_pd13
                          , table_name='supply'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['amount']
                          , float_columns=['price']
                          )

df_pd13

,supply_id,title,author,price,amount
1,1,Доктор Живаго,Пастернак Б.Л.,618.99,3
2,2,Черный человек,Есенин С.А.,570.20,6
3,3,Евгений Онегин,Пушкин А.С.,440.80,5
4,4,Идиот,Достоевский Ф.М.,360.80,3


In [43]:
sql_1292 = """SELECT 
    book.title --AS Название
    , author.name_author --AS Автор
    , (supply.amount + book.amount) --AS Количество

FROM book

INNER JOIN author USING (author_id)  

INNER JOIN supply 
ON book.title = supply.title 
and book.price = supply.price
;"""

spark.sql(sql_1292).toPandas()

,title,name_author,(amount + amount)
0,Черный человек,Есенин С.А.,12


### Задание_13


In [44]:
sql_1293 = """select

    dd.*
    , cc.name_city

from( select
         gg.name_genre
         , aa.name_author
         , bb.title
         , CAST( (FLOOR(RAND()*2)+1) as INTEGER) as city_id
     
     from book as bb

     left join author as aa
     on bb.author_id = aa.author_id

     left join genre as gg
     on bb.genre_id = gg.genre_id
     
) as dd

left join city as cc
on dd.city_id*1 = cc.city_id*1

ORDER BY 4,5
;"""

spark.sql(sql_1293).toPandas()

,name_genre,name_author,title,city_id,name_city
0,Роман,Булгаков М.А.,Мастер и Маргарита,1,Москва
1,Приключения,Булгаков М.А.,Белая гвардия,1,Москва
2,Роман,Есенин С.А.,Черный человек,2,Санкт-Петербург
3,Роман,Достоевский Ф.М.,Идиот,2,Санкт-Петербург
4,Роман,Достоевский Ф.М.,Братья Карамазовы,2,Санкт-Петербург
5,Роман,Достоевский Ф.М.,Игрок,2,Санкт-Петербург
6,Поэзия,Есенин С.А.,Стихотворения и поэмы,2,Санкт-Петербург
7,Поэзия,Пастернак Б.Л.,Лирика,2,Санкт-Петербург


## 2.3 Запросы корректировки, соединение таблиц

## Практика

In [45]:
# Создадим таблицу author

t11 = """author_id	name_author
1	Булгаков М.А.
2	Достоевский Ф.М.
3	Есенин С.А.
4	Пастернак Б.Л.
5	Лермонтов М.Ю."""

df_pd14 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t11.split('\n')] )#.drop([0,7],axis=1)
 
df_pd14.columns = df_pd14.iloc[0,:]
df_pd14 = df_pd14.iloc[1:,:]

mySchema = StructType([ T.StructField("author_id", T.IntegerType(), True)\
                       , T.StructField("name_author", T.StringType(), True)
                       ])
df_sp14, df_pd14 = pDF_to_sDF( df_pd14
                          , table_name='author'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd14

,author_id,name_author
1,1,Булгаков М.А.
2,2,Достоевский Ф.М.
3,3,Есенин С.А.
4,4,Пастернак Б.Л.
5,5,Лермонтов М.Ю.


In [46]:
# Создадим таблицу genre

t12 = """genre_id	name_genre
1	Роман
2	Поэзия
3	Приключения"""

df_pd15 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t12.split('\n')] )#.drop([0,7],axis=1)
 
df_pd15.columns = df_pd15.iloc[0,:]
df_pd15 = df_pd15.iloc[1:,:]

mySchema = StructType([ T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("name_genre", T.StringType(), True)
                       ])
df_sp15, df_pd15 = pDF_to_sDF( df_pd15
                          , table_name='genre'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd15

,genre_id,name_genre
1,1,Роман
2,2,Поэзия
3,3,Приключения


In [47]:
# Создадим таблицу book

t13 = """book_id	title	author_id	genre_id	price	amount
1	Мастер и Маргарита	1	1	670.99	3
2	Белая гвардия	1	1	540.50	5
3	Идиот	2	1	460.00	10
4	Братья Карамазовы	2	1	799.01	3
5	Игрок	2	1	480.50	10
6	Стихотворения и поэмы	3	2	650.00	15
7	Черный человек	3	2	570.20	6
8	Лирика	4	2	518.99	2"""

df_pd16 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t13.split('\n')] )#.drop([0,7],axis=1)
 
df_pd16.columns = df_pd16.iloc[0,:]
df_pd16 = df_pd16.iloc[1:,:]

mySchema = StructType([ T.StructField("book_id", T.IntegerType(), True)\
                       , T.StructField("title", T.StringType(), True)\
                       , T.StructField("author_id", T.IntegerType(), True)\
                       , T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("price", T.FloatType(), True)\
                       , T.StructField("amount", T.IntegerType(), True)
                       ])
df_sp16, df_pd16 = pDF_to_sDF( df_pd16
                          , table_name='book'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['amount']
                          , float_columns=['price']
                          )

df_pd16

,book_id,title,author_id,genre_id,price,amount
1,1,Мастер и Маргарита,1,1,670.99,3
2,2,Белая гвардия,1,1,540.50,5
3,3,Идиот,2,1,460.00,10
4,4,Братья Карамазовы,2,1,799.01,3
5,5,Игрок,2,1,480.50,10
6,6,Стихотворения и поэмы,3,2,650.00,15
7,7,Черный человек,3,2,570.20,6
8,8,Лирика,4,2,518.99,2


In [48]:
# Создадим таблицу supply

t14 = """supply_id	title	author	price	amount
1	Доктор Живаго	Пастернак Б.Л.	380.80	4
2	Черный человек 	Есенин С.А.	570.20	6
3	Белая гвардия	Булгаков М.А.	540.50	7
4	Идиот	Достоевский Ф.М.	360.80	3
5	Стихотворения и поэмы	Лермонтов М.Ю.	255.90	4
6	Остров сокровищ	Стивенсон Р.Л.	599.99	5"""

df_pd17 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t14.split('\n')] )#.drop([0,7],axis=1)
 
df_pd17.columns = df_pd17.iloc[0,:]
df_pd17 = df_pd17.iloc[1:,:]

mySchema = StructType([ T.StructField("supply_id", T.IntegerType(), True)\
                       , T.StructField("title", T.StringType(), True)\
                       , T.StructField("author", T.StringType(), True)\
                       , T.StructField("price", T.FloatType(), True)\
                       , T.StructField("amount", T.IntegerType(), True)
                       ])
df_sp17, df_pd17 = pDF_to_sDF( df_pd17
                          , table_name='supply'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['amount']
                          , float_columns=['price']
                          )

df_pd17

,supply_id,title,author,price,amount
1,1,Доктор Живаго,Пастернак Б.Л.,380.80,4
2,2,Черный человек,Есенин С.А.,570.20,6
3,3,Белая гвардия,Булгаков М.А.,540.50,7
4,4,Идиот,Достоевский Ф.М.,360.80,3
5,5,Стихотворения и поэмы,Лермонтов М.Ю.,255.90,4
6,6,Остров сокровищ,Стивенсон Р.Л.,599.99,5


### Задание_1
<p>Для книг, которые уже есть на складе (в таблице<strong><code> book</code></strong>), но по другой цене, чем&nbsp;в поставке (<code><strong>supply</strong></code>),&nbsp; необходимо в таблице <strong><code>book&nbsp;</code></strong>увеличить количество на значение, указанное в поставке,&nbsp; и пересчитать цену. А в&nbsp;таблице&nbsp; <code><strong>supply&nbsp;</strong></code>обнулить&nbsp;количество этих книг. Формула для пересчета цены:</p>

price = (p1*k1+p2*k2)/(k1+k2)

<p>где&nbsp; <em>p<sub>1</sub>, p<sub>2</sub></em> - цена книги в таблицах <strong><code>book&nbsp;</code></strong>и&nbsp;<code><strong>supply</strong></code>;</p>

<p>&nbsp; &nbsp; &nbsp; &nbsp;<em>k<sub>1</sub>, k<sub>2</sub> </em>- количество книг в таблицах <strong><code>book&nbsp;</code></strong>и&nbsp;<code><strong>supply</strong></code>.</p>

In [49]:
sql_230 = """UPDATE book 
     INNER JOIN author 
        ON author.author_id = book.author_id
     INNER JOIN supply 
        ON book.title = supply.title 
        and supply.author = author.name_author
SET book.price = (book.price*book.amount+supply.price*supply.amount) / (book.amount+supply.amount),
    book.amount = book.amount + supply.amount,
    supply.amount = 0
WHERE book.price <> supply.price;

SELECT * FROM book;

SELECT * FROM supply;"""

sprint(sql_230)

### Задание_2
<p>Включить новых авторов в таблицу <code><strong>author</strong></code> с помощью запроса на добавление, а затем вывести все данные из таблицы <code><strong>author</strong></code>.&nbsp; Новыми считаются авторы, которые есть в таблице <code><strong>supply</strong></code>, но нет в таблице <code><strong>author</strong></code>.</p>

In [50]:
sql_231 = """INSERT INTO author (name_author)
  SELECT
    supply.author
  FROM supply
  LEFT JOIN  author 
    ON author.name_author = supply.author
  WHERE author.name_author is NULL
;

SELECT * FROM author;"""

sprint(sql_231)

### Задание_3

<p>Добавить новые книги из таблицы <code><strong>supply</strong></code> в таблицу <code><strong>book</strong></code> на основе сформированного выше запроса. Затем вывести для просмотра таблицу <code><strong>book</strong></code>.</p>

In [51]:
sql_232 = """INSERT INTO book (title,author_id,genre_id,price,amount)
SELECT  supply.title
        , author.author_id
        , NULL AS genre_id
        , supply.price
        , supply.amount
FROM  author 
INNER JOIN supply 
    ON author.name_author = supply.author
WHERE amount <> 0;

SELECT * FROM book;"""

sprint(sql_232)

### Задание_4
<p>&nbsp;Занести для книги «Стихотворения и поэмы» Лермонтова жанр «Поэзия», а для книги «Остров&nbsp;сокровищ» Стивенсона - «Приключения». (Использовать два&nbsp;запроса).</p>

In [52]:
sql_233="""UPDATE book
SET genre_id = 
      (
       SELECT genre_id 
       FROM genre 
       WHERE name_genre = 'Поэзия'
      )
WHERE title = 'Стихотворения и поэмы'
    AND genre_id IS NULL;

UPDATE book
SET genre_id = 
      (
       SELECT genre_id 
       FROM genre 
       WHERE name_genre = 'Приключения'
      )
WHERE title = 'Остров сокровищ'
    AND genre_id IS NULL;

SELECT * FROM book;"""

sprint(sql_233)

### Задание_5
Удалить всех авторов и все их книги, общее количество книг которых меньше 20.

In [53]:
sql_234="""DELETE FROM author
WHERE author_id IN (
        SELECT
            author_id
        FROM book
        GROUP BY author_id
        HAVING SUM(amount)<20
);

SELECT * FROM author;

SELECT * FROM book;"""

sprint(sql_234)

### Задание_6
<p>Удалить все жанры, к которым относится меньше 4-х книг. В таблице <code><strong>book</strong></code> для этих жанров установить значение <code>Null</code>.</p>

In [54]:
sql_235="""DELETE FROM genre
WHERE genre_id IN (
        SELECT
            genre_id
        FROM book
        GROUP BY genre_id
        HAVING COUNT(DISTINCT book_id)<4
);

SELECT * FROM genre;

SELECT * FROM book;"""

sprint(sql_235)

### Задание_7
<p>Удалить всех авторов, которые пишут&nbsp;в жанре "Поэзия". Из таблицы <code><strong>book</strong></code> удалить все книги этих авторов. В запросе для отбора авторов использовать полное название жанра, а не его <code><strong>id</strong></code>.</p>

In [55]:
sql_236="""DELETE FROM author
USING 
    author 
    INNER JOIN book ON author.author_id = book.author_id
    INNER JOIN genre ON genre.genre_id = book.genre_id
WHERE genre.name_genre = 'Поэзия';

SELECT * FROM author;

SELECT * FROM book;"""

sprint(sql_236)

## 2.4 База данных «Интернет-магазин книг», запросы на выборку

<p>В интернет-магазине&nbsp;продаются книги. Каждая книга имеет название, написана одним автором,&nbsp;относится к одному жанру, имеет определенную цену. В магазине в наличии&nbsp;есть несколько экземпляров каждой книги.&nbsp;</p>

<p>Покупатель регистрируется на сайте интернет-магазина, задает свое имя и фамилию,&nbsp;электронную почту<em>&nbsp;</em>и город проживания. Он может сформировать один или несколько заказов<em>,&nbsp;</em>для каждого заказа написать какие-то пожелания.&nbsp;Каждый заказ включает&nbsp;одну или несколько книг, каждую книгу можно заказать в нескольких экземплярах. Затем&nbsp;заказ проходит ряд последовательных этапов (операций): оплачивается, упаковывается, передается курьеру или транспортной компании для транспортировки и, наконец, доставляется покупателю. Фиксируется дата каждой операции.&nbsp;Для каждого города известно среднее время доставки книг.</p>

<p>При этом в магазине&nbsp;ведется учет книг, при покупке их количество уменьшается, при поступлении товара увеличивается, при исчерпании количества – оформляется заказ&nbsp;и пр.</p>

<p>В данном уроке сначала будет построена <a href="https://stepik.org/lesson/308891/step/2?unit=291017" rel="noopener noreferrer nofollow">концептуальная модель</a> базы данных, затем ее <a href="https://stepik.org/lesson/308891/step/3?unit=291017" rel="noopener noreferrer nofollow">логическая модель</a>. Также будут определены<a href="https://stepik.org/lesson/308891/step/4?unit=291017" rel="noopener noreferrer nofollow"> структура и содержание&nbsp;таблиц</a> базы данных «Интернет-магазин книг».</p>

<p><img alt="" src="https://ucarecdn.com/bad26356-5e34-4945-a9d4-0748686a6b54/">​</p>

## Практика

<details open=""><summary><strong>Наполнение таблиц</strong></summary> <img alt="" height="870" name="111.png" src="https://ucarecdn.com/09eb90ce-ce66-446b-8337-e391caabff1c/" width="719"></details>

In [56]:
# Создадим таблицу author
# Создадим таблицу genre
# Создадим таблицу book

In [57]:
t20 = """1	Баранов Павел	3	baranov@test
2	Абрамова Катя	1	abramova@test
3	Семенонов Иван	2	semenov@test
4	Яковлева Галина	1	yakovleva@test"""

df_pd20 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t20.split('\n')] )
 
df_pd20.columns = 'client_id	name_client	city_id	email'.split('\t')

mySchema = StructType([
                       T.StructField("client_id", T.IntegerType(), True)\
                       , T.StructField("name_client", T.StringType(), True)
                       , T.StructField("city_id", T.IntegerType(), True)
                       , T.StructField("email", T.StringType(), True)
                       ])
df_sp20, df_pd20 = pDF_to_sDF( df_pd20
                          , table_name='client'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd20

,client_id,name_client,city_id,email
0,1,Баранов Павел,3,baranov@test
1,2,Абрамова Катя,1,abramova@test
2,3,Семенонов Иван,2,semenov@test
3,4,Яковлева Галина,1,yakovleva@test


In [58]:
t21 = """1	Доставка только вечером	1
2	 	3
3	Упаковать каждую книгу по отдельности	2
4	 	1"""

df_pd21 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t21.split('\n')] )
 
df_pd21.columns = 'buy_id	buy_description	client_id'.split('\t')

mySchema = StructType([
                       T.StructField("buy_id", T.IntegerType(), True)\
                       , T.StructField("buy_description", T.StringType(), True)
                       , T.StructField("client_id", T.IntegerType(), True)
                       ])
df_sp21, df_pd21 = pDF_to_sDF( df_pd21
                          , table_name='buy'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd21

,buy_id,buy_description,client_id
0,1,Доставка только вечером,1
1,2,,3
2,3,Упаковать каждую книгу по отдельности,2
3,4,,1


In [59]:
t22="""1	1	1	1
2	1	7	2
3	1	3	1
4	2	8	2
5	3	3	2
6	3	2	1
7	3	1	1
8	4	5	1"""

df_pd22 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t22.split('\n')] )
 
df_pd22.columns = 'buy_book_id	buy_id	book_id	amount'.split('\t')

mySchema = StructType([
                       T.StructField("buy_book_id", T.IntegerType(), True)\
                       , T.StructField("buy_id", T.IntegerType(), True)\
                       , T.StructField("book_id", T.IntegerType(), True)\
                       , T.StructField("amount", T.StringType(), True)
                       ])
df_sp22, df_pd22 = pDF_to_sDF( df_pd22
                          , table_name='buy_book'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          , int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd22

,buy_book_id,buy_id,book_id,amount
0,1,1,1,1
1,2,1,7,2
2,3,1,3,1
3,4,2,8,2
4,5,3,3,2
5,6,3,2,1
6,7,3,1,1
7,8,4,5,1


In [60]:
t23="""1	Оплата
2	Упаковка
3	Транспортировка
4	Доставка"""

df_pd23 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t23.split('\n')] )
 
df_pd23.columns = 'step_id	name_step'.split('\t')

mySchema = StructType([
                       T.StructField("step_id", T.IntegerType(), True)\
                       , T.StructField("name_step", T.StringType(), True)
                       ])
df_sp23, df_pd23 = pDF_to_sDF( df_pd23
                          , table_name='step'
                          , mySchema=mySchema
                          #, date_columns=['date_violation','date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )
df_pd23

,step_id,name_step
0,1,Оплата
1,2,Упаковка
2,3,Транспортировка
3,4,Доставка


In [61]:
t24="""1	 1	1	2020-02-20	2020-02-20
2	1	2	2020-02-20	2020-02-21
3	1	3	2020-02-22	2020-03-07
4	1	4	2020-03-08	2020-03-08
5	2	1	2020-02-28	2020-02-28
6	2	2	2020-02-29	2020-03-01
7	2	3	2020-03-02	 
8	2	4	 	 
9	3	1	2020-03-05	2020-03-05
10	3	2	2020-03-05	2020-03-06
11	3	3	2020-03-06	2020-03-10
12	3	4	2020-03-11	 
13	4	1	2020-03-20	 
14	4	2	 	 
15	4	3	 	 
16	4	4	 	 """

df_pd24 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t24.split('\n')] )
 
df_pd24.columns = 'buy_step_id	buy_id	step_id	date_step_beg	date_step_end'.split('\t')

mySchema = StructType([
                       T.StructField("buy_step_id", T.IntegerType(), True)\
                       , T.StructField("buy_id", T.IntegerType(), True)\
                       , T.StructField("step_id", T.IntegerType(), True)\
                       , T.StructField("date_step_beg", T.TimestampType(), True)\
                       , T.StructField("date_step_end", T.TimestampType(), True)
                       ])
df_sp24, df_pd24 = pDF_to_sDF( df_pd24
                          , table_name='buy_step'
                          , mySchema=mySchema
                          , date_columns=['date_step_beg','date_step_end']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )
df_pd24

,buy_step_id,buy_id,step_id,date_step_beg,date_step_end
0,1,1,1,2020-02-20,2020-02-20
1,2,1,2,2020-02-20,2020-02-21
2,3,1,3,2020-02-22,2020-03-07
3,4,1,4,2020-03-08,2020-03-08
4,5,2,1,2020-02-28,2020-02-28
5,6,2,2,2020-02-29,2020-03-01
6,7,2,3,2020-03-02,NaT
7,8,2,4,NaT,NaT
8,9,3,1,2020-03-05,2020-03-05
9,10,3,2,2020-03-05,2020-03-06


In [62]:
# Создадим таблицу city

t25 = """1	Москва	5
2	Санкт-Петербург	3
3	Владивосток	12"""

df_pd25 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t25.split('\n')] )
 
df_pd25.columns = 'city_id	name_city	days_delivery'.split('\t')

mySchema = StructType([
                       T.StructField("city_id", T.IntegerType(), True)\
                       , T.StructField("name_city", T.StringType(), True)\
                       , T.StructField("days_delivery", T.IntegerType(), True)
                       ])
df_sp25, df_pd25 = pDF_to_sDF( df_pd25
                          , table_name='city'
                          , mySchema=mySchema
                          #, date_columns=['date_step_beg','date_step_end']
                          , int_columns=['days_delivery']
                          #, float_columns=['price']
                          )

df_pd25

,city_id,name_city,days_delivery
0,1,Москва,5
1,2,Санкт-Петербург,3
2,3,Владивосток,12


### Задание_1
Вывести все заказы Баранова Павла (id заказа, какие книги, по какой цене и в каком количестве он заказал) в отсортированном по номеру заказа и названиям книг виде.

In [63]:
sql_240="""SELECT 
    buy_book.buy_id
    , book.title
    , CAST( book.price AS DECIMAL(18,2) ) AS price
    , buy_book.amount

FROM client 
    INNER JOIN buy ON client.client_id = buy.client_id
    INNER JOIN buy_book ON buy_book.buy_id = buy.buy_id
    INNER JOIN book ON buy_book.book_id=book.book_id
WHERE 
    client.name_client = 'Баранов Павел' 
    
ORDER BY buy_book.buy_id
    , book.title
;"""

spark.sql(sql_240).toPandas()

,buy_id,title,price,amount
0,1,Идиот,460.00,1.0
1,1,Мастер и Маргарита,670.99,1.0
2,1,Черный человек,570.20,2.0
3,4,Игрок,480.50,1.0


### Задание_2
<p>Посчитать, сколько раз была заказана каждая книга, для книги вывести ее автора (нужно посчитать, в каком количестве заказов фигурирует каждая книга).&nbsp; Вывести фамилию и инициалы автора, название книги,&nbsp;последний столбец назвать&nbsp;<code><strong>Количество</strong></code>.&nbsp;Результат отсортировать сначала&nbsp;&nbsp;по фамилиям авторов, а потом по названиям книг.</p>

<p><strong>Фрагмент логической схемы базы данных:</strong></p>

<p><img alt="" src="https://ucarecdn.com/c9356f96-1b19-42ce-8c9e-e7a8169734fc/"></p>

In [64]:
sql_241="""SELECT 
    author.name_author
    , book.title
    , COUNT(DISTINCT buy_book.buy_id) AS Kolichestvo

FROM author 
INNER JOIN book ON author.author_id=book.author_id
LEFT JOIN buy_book ON buy_book.book_id = book.book_id

GROUP BY 
    author.name_author
    , book.title
ORDER BY 
    author.name_author
    , book.title
;"""

spark.sql(sql_241).toPandas()

,name_author,title,Kolichestvo
0,Булгаков М.А.,Белая гвардия,1
1,Булгаков М.А.,Мастер и Маргарита,2
2,Достоевский Ф.М.,Братья Карамазовы,0
3,Достоевский Ф.М.,Игрок,1
4,Достоевский Ф.М.,Идиот,2
5,Есенин С.А.,Стихотворения и поэмы,0
6,Есенин С.А.,Черный человек,1
7,Пастернак Б.Л.,Лирика,1


### Задание_3
Вывести города, в которых живут клиенты, оформлявшие заказы в интернет-магазине. Указать количество заказов в каждый город, этот столбец назвать **Количество**. Информацию вывести по убыванию количества заказов, а затем в алфавитном порядке по названию городов.

In [65]:
sql_242="""SELECT 
    city.name_city
    , COUNT(DISTINCT buy.buy_id ) AS Kolichestvo

FROM city 
LEFT JOIN client ON city.city_id=client.city_id
LEFT JOIN buy ON client.client_id = buy.client_id

GROUP BY city.name_city

ORDER BY Kolichestvo DESC
    , city.name_city ASC
;"""

spark.sql(sql_242).toPandas()

,name_city,Kolichestvo
0,Владивосток,2
1,Москва,1
2,Санкт-Петербург,1


### Задание_4
Вывести номера всех оплаченных заказов и даты, когда они были оплачены.

In [66]:
sql_243="""SELECT 
    buy_step.buy_id
    , buy_step.date_step_end

FROM buy_step 
INNER JOIN step ON buy_step.step_id=step.step_id
    AND step.name_step = 'Оплата'

WHERE buy_step.date_step_end IS NOT NULL
;"""

spark.sql(sql_243).toPandas()

,buy_id,date_step_end
0,1,2020-02-20
1,2,2020-02-28
2,3,2020-03-05


### Задание_5


In [67]:
sql_244="""SELECT 
    buy_book.buy_id
    , client.name_client
    , CAST( SUM( buy_book.amount*book.price ) AS DECIMAL(18,2)) AS Stoimost

FROM buy_book
INNER JOIN buy ON buy_book.buy_id=buy.buy_id
INNER JOIN client ON client.client_id=buy.client_id
INNER JOIN book ON buy_book.book_id=book.book_id

GROUP BY 
    buy_book.buy_id
    , client.name_client
    
ORDER BY 
    buy_book.buy_id
;"""

spark.sql(sql_244).toPandas()

,buy_id,name_client,Stoimost
0,1,Баранов Павел,2271.39
1,2,Семенонов Иван,1037.98
2,3,Абрамова Катя,2131.49
3,4,Баранов Павел,480.50


### Задание_6
<p>Вывести номера заказов (<code><strong>buy_id</strong></code>)&nbsp;и названия&nbsp;этапов, &nbsp;на которых они в данный момент находятся. Если заказ доставлен –&nbsp; информацию о нем не выводить. Информацию отсортировать по возрастанию <code><strong>buy_id</strong></code>.</p>

In [68]:
sql_245="""SELECT

    buy_step.buy_id
    , step.name_step

    --, buy_step.date_step_beg
    --, buy_step.date_step_end

FROM buy_step
INNER JOIN step ON buy_step.step_id=step.step_id

--INNER JOIN buy_book ON buy_book.buy_id=buy_step.buy_id
--INNER JOIN buy ON buy_book.buy_id=buy.buy_id
--INNER JOIN client ON client.client_id=buy.client_id
--INNER JOIN book ON buy_book.book_id=book.book_id

-- Выберем последний статус каждого ЗАКАЗА
INNER JOIN (
        SELECT
            buy_id
            , MAX(date_step_beg) AS date_step_beg
        FROM buy_step
        WHERE date_step_beg IS NOT NULL
              AND date_step_end IS NULL
        GROUP BY buy_id
       ) AS dd
    ON dd.buy_id = buy_step.buy_id
    AND dd.date_step_beg = buy_step.date_step_beg

-- Уберем ЗАКАЗЫ в статусе ДОСТАВЛЕН
WHERE buy_step.buy_id IN (
        SELECT
            buy_step.buy_id
        FROM buy_step
        WHERE buy_step.step_id = 4
        AND buy_step.date_step_end IS NULL
        )

ORDER BY 
    buy_step.buy_id
    --, step.step_id
;"""

spark.sql(sql_245).toPandas()

,buy_id,name_step
0,2,Транспортировка
1,3,Доставка
2,4,Оплата


In [69]:
sprint(sql_245)

### Задание_7
<p>В таблице <code><strong>city</strong></code> для каждого города указано количество дней, за которые заказ может быть доставлен в этот город (рассматривается только этап <strong>Транспортировка</strong>). Для тех заказов, которые прошли этап транспортировки, вывести количество дней за которое заказ реально доставлен в город. А также, если заказ доставлен с опозданием, указать количество дней задержки, в противном случае вывести 0. В результат включить номер заказа (<code><strong>buy_id</strong></code>), а также вычисляемые столбцы <code><strong>Количество_дней</strong></code> и <code><strong>Опоздание</strong></code>. Информацию вывести в отсортированном по номеру заказа виде.</p>

In [70]:
sql_245="""SELECT
    buy_step.buy_id
    --, step.name_step

    --, buy_step.date_step_beg
    --, buy_step.date_step_end
    
    --, city.name_city
    --, city.days_delivery
    
    , DATEDIFF(buy_step.date_step_end, buy_step.date_step_beg) AS Kolichestvo_Dney 
    , IF(
        DATEDIFF(buy_step.date_step_end, buy_step.date_step_beg) > city.days_delivery
        , DATEDIFF(buy_step.date_step_end, buy_step.date_step_beg) - city.days_delivery
        , 0
    ) AS Opozdanie

FROM buy_step
INNER JOIN step ON buy_step.step_id=step.step_id AND step.name_step='Транспортировка'
INNER JOIN buy ON buy_step.buy_id=buy.buy_id
INNER JOIN client ON client.client_id=buy.client_id
INNER JOIN city ON client.city_id=city.city_id

--INNER JOIN buy_book ON buy_book.buy_id=buy_step.buy_id
--INNER JOIN book ON buy_book.book_id=book.book_id

WHERE buy_step.date_step_end IS NOT NULL

ORDER BY 
    buy_step.buy_id
;"""

spark.sql(sql_245).toPandas()

,buy_id,Kolichestvo_Dney,Opozdanie
0,1,14,2
1,3,4,0


### Задание_8
<p>Выбрать всех клиентов, которые заказывали книги Достоевского, информацию вывести в отсортированном по алфавиту виде. В решении используйте фамилию автора, а не его <code><strong>id</strong></code>.</p>

In [71]:
sql_246="""SELECT DISTINCT

    client.name_client
    --, buy_step.buy_id
    --, step.name_step

    --, buy_step.date_step_beg
    --, buy_step.date_step_end
    
    --, city.name_city
    --, city.days_delivery
    
FROM buy_book
INNER JOIN buy ON buy_book.buy_id=buy.buy_id
INNER JOIN client ON client.client_id=buy.client_id
INNER JOIN book ON buy_book.book_id=book.book_id
INNER JOIN author ON author.author_id=book.author_id

--INNER JOIN buy_step ON buy_book.buy_id=buy_step.buy_id
--INNER JOIN step ON buy_step.step_id=step.step_id AND step.name_step='Транспортировка'
--INNER JOIN city ON client.city_id=city.city_id


WHERE author.name_author LIKE '%Достоевский%'

ORDER BY 
    client.name_client
;"""

spark.sql(sql_246).toPandas()

,name_client
0,Абрамова Катя
1,Баранов Павел


### Задание_9
<p>Вывести жанр (или жанры), в котором было заказано больше всего экземпляров книг, указать это количество. Последний столбец назвать <strong>Количество</strong>.</p>

In [72]:
sql_247="""SELECT

    genre.name_genre
    , CAST( SUM(buy_book.amount) AS INT) AS Kolichestvo
    
    --, buy_step.buy_id
    --, step.name_step

    --, buy_step.date_step_beg
    --, buy_step.date_step_end
    
    --, city.name_city
    --, city.days_delivery
    
FROM buy_book
INNER JOIN book ON buy_book.book_id=book.book_id
INNER JOIN author ON author.author_id=book.author_id
INNER JOIN genre ON genre.genre_id=book.genre_id

--INNER JOIN buy ON buy_book.buy_id=buy.buy_id
--INNER JOIN client ON client.client_id=buy.client_id
--INNER JOIN buy_step ON buy_book.buy_id=buy_step.buy_id
--INNER JOIN step ON buy_step.step_id=step.step_id AND step.name_step='Транспортировка'
--INNER JOIN city ON client.city_id=city.city_id

GROUP BY 
    genre.name_genre

ORDER BY 
    Kolichestvo DESC
LIMIT 1
;"""

spark.sql(sql_247).toPandas()

,name_genre,Kolichestvo
0,Роман,7


<p><strong>Оператор UNION</strong></p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> столбец_1_1, столбец_1_2, ...
<span class="hljs-keyword">FROM</span> 
  ...
<span class="hljs-keyword">UNION</span>
<span class="hljs-keyword">SELECT</span> столбец_2_1, столбец_2_2, ...
<span class="hljs-keyword">FROM</span> 
  ...</code></pre>

<p>или</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> столбец_1_1, столбец_1_2, ...
<span class="hljs-keyword">FROM</span> 
&nbsp; ...
<span class="hljs-keyword">UNION</span> ALL
<span class="hljs-keyword">SELECT</span> столбец_2_1, столбец_2_2, ...
<span class="hljs-keyword">FROM</span> 
&nbsp; ...</code></pre>

<p>В результате выполнения этой конструкции будет выведена таблица, имена столбцов которой соответствуют именам столбцов в первом запросе. А в таблице результата сначала отображаются записи-результаты первого запроса, а затем второго. Если указано ключевое слово <code><strong>ALL</strong></code>, то в результат включаются все записи запросов, в противном случае - различные.</p>

<p>С <code><strong>UNION </strong></code>клиенты будут выведены без повторений.</p>

In [73]:
t26="""| buy_archive_id | buy_id | client_id | book_id | date_payment | price  | amount |
| 1              | 2      | 1         | 1       | 2019-02-21   | 670.60 | 2      |
| 2              | 2      | 1         | 3       | 2019-02-21   | 450.90 | 1      |
| 3              | 1      | 2         | 2       | 2019-02-10   | 520.30 | 2      |
| 4              | 1      | 2         | 4       | 2019-02-10   | 780.90 | 3      |
| 5              | 1      | 2         | 3       | 2019-02-10   | 450.90 | 1      |
| 6              | 3      | 4         | 4       | 2019-03-05   | 780.90 | 4      |
| 7              | 3      | 4         | 5       | 2019-03-05   | 480.90 | 2      |
| 8              | 4      | 1         | 6       | 2019-03-12   | 650.00 | 1      |
| 9              | 5      | 2         | 1       | 2019-03-18   | 670.60 | 2      |
| 10             | 5      | 2         | 4       | 2019-03-18   | 780.90 | 1      |"""

df_pd26 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("|")] for x in t26.split('\n')] ).drop([0,8],axis=1)
 

df_pd26.columns = df_pd26.iloc[0,:]
df_pd26 = df_pd26.iloc[1:,:]

mySchema = StructType([
                       T.StructField("buy_archive_id", T.IntegerType(), True)\
                       , T.StructField("buy_id", T.IntegerType(), True)\
                       , T.StructField("client_id", T.IntegerType(), True)\
                       , T.StructField("book_id", T.IntegerType(), True)\
                       , T.StructField("date_payment", T.TimestampType(), True)\
                       , T.StructField("price", T.FloatType(), True)\
                       , T.StructField("amount", T.IntegerType(), True)
                       ])
df_sp26, df_pd26 = pDF_to_sDF( df_pd26
                          , table_name='buy_archive'
                          , mySchema=mySchema
                          , date_columns=['date_payment']
                          , int_columns=['amount']
                          , float_columns=['price']
                          )

df_pd26

,buy_archive_id,buy_id,client_id,book_id,date_payment,price,amount
1,1,2,1,1,2019-02-21,670.6,2
2,2,2,1,3,2019-02-21,450.9,1
3,3,1,2,2,2019-02-10,520.3,2
4,4,1,2,4,2019-02-10,780.9,3
5,5,1,2,3,2019-02-10,450.9,1
6,6,3,4,4,2019-03-05,780.9,4
7,7,3,4,5,2019-03-05,480.9,2
8,8,4,1,6,2019-03-12,650.0,1
9,9,5,2,1,2019-03-18,670.6,2
10,10,5,2,4,2019-03-18,780.9,1


### Задание_10

In [74]:
sql_248="""
SELECT 
    --buy_id
    --, client_id
    --, book_id
    
    YEAR(date_payment) AS Year
    , MONTH(date_payment) AS Month
    
    --, amount
    --, price
    
    , CAST( SUM(amount*price) AS DECIMAL(18,2)) AS Summa
    
FROM  buy_archive

GROUP BY Year, Month

UNION ALL

SELECT
    --buy.buy_id
    --, client_id
    --, book_id
    
    YEAR(date_step_end) AS Year
    , MONTH(date_step_end) AS Month
    
    --, buy_book.amount
    --, price
    
    , CAST( SUM(buy_book.amount*price) AS DECIMAL(18,2)) AS Summa
    
FROM  book 
INNER JOIN buy_book USING(book_id)
INNER JOIN buy USING(buy_id) 
INNER JOIN buy_step USING(buy_id)
INNER JOIN step USING(step_id)
    
WHERE  date_step_end IS NOT Null AND name_step = "Оплата"

GROUP BY Year, Month

ORDER BY Month, Year
;"""

spark.sql(sql_248).toPandas()

,Year,Month,Summa
0,2019,2,5626.30
1,2020,2,3309.37
2,2019,3,6857.50
3,2020,3,2131.49


### Задание_11
<p>Для каждой отдельной книги необходимо вывести информацию о количестве проданных экземпляров и их стоимости за 2020 и 2019 год . Вычисляемые столбцы&nbsp;назвать <code><strong>Количество</strong></code><strong>&nbsp;</strong>и&nbsp;<code><strong>Сумма</strong></code>. Информацию отсортировать по убыванию стоимости.</p>

In [75]:
sql_249="""SELECT

    book.title
    --, tt.God
    , CAST( SUM(tt.Kolichestvo) AS INT) AS Kolichestvo
    , CAST( SUM(tt.Summa) AS DECIMAL(18,2)) AS Summa

FROM( SELECT 
        --buy_id
        --, client_id
        book_id

        , YEAR(date_payment) AS God
        --, MONTHNAME(date_payment) AS Mesyac

        , SUM(amount) AS Kolichestvo
        --, price

        , SUM(amount*price) AS Summa

    FROM  buy_archive

    GROUP BY book_id
        , God
        --, Mesyac

    UNION ALL 

    SELECT
        --buy.buy_id
        --, client_id
        book_id

        , YEAR(date_step_end) AS God
        --, MONTHNAME(date_step_end) AS Mesyac

        , SUM(buy_book.amount) AS Kolichestvo
        --, price

        , SUM(buy_book.amount*price) AS Summa

    FROM  book 
    INNER JOIN buy_book USING(book_id)
    INNER JOIN buy USING(buy_id) 
    INNER JOIN buy_step USING(buy_id)
    INNER JOIN step USING(step_id)

    WHERE  date_step_end IS NOT Null 
        AND name_step = "Оплата"


    GROUP BY book_id
            , God
            --, Mesyac
) AS tt

LEFT JOIN book USING(book_id)

WHERE tt.God IN (2019, 2020)

GROUP BY book.title

ORDER BY Summa DESC
;
"""

spark.sql(sql_249).toPandas()

,title,Kolichestvo,Summa
0,Братья Карамазовы,8,6247.20
1,Мастер и Маргарита,6,4024.38
2,Идиот,5,2281.80
3,Белая гвардия,3,1581.10
4,Черный человек,2,1140.40
5,Лирика,2,1037.98
6,Игрок,2,961.80
7,Стихотворения и поэмы,1,650.00


## 2.5 База данных «Интернет-магазин книг», запросы корректировки

## Практика

In [76]:
t27 = """1	Баранов Павел	3	baranov@test
2	Абрамова Катя	1	abramova@test
3	Семенонов Иван	2	semenov@test
4	Яковлева Галина	1	yakovleva@test
5	Попов Илья	1	popov@test"""

df_pd27 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t27.split('\n')] )
 
df_pd27.columns = 'client_id	name_client	city_id	email'.split('\t')

mySchema = StructType([
                       T.StructField("client_id", T.IntegerType(), True)\
                       , T.StructField("name_client", T.StringType(), True)
                       , T.StructField("city_id", T.IntegerType(), True)
                       , T.StructField("email", T.StringType(), True)
                       ])
df_sp27, df_pd27 = pDF_to_sDF( df_pd27
                          , table_name='client'
                          , mySchema=mySchema
                          #, date_columns=['date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd27

,client_id,name_client,city_id,email
0,1,Баранов Павел,3,baranov@test
1,2,Абрамова Катя,1,abramova@test
2,3,Семенонов Иван,2,semenov@test
3,4,Яковлева Галина,1,yakovleva@test
4,5,Попов Илья,1,popov@test


In [77]:
t28 = """1	Доставка только вечером	1
2	 	3
3	Упаковать каждую книгу по отдельности	2
4	 	1
5	Связаться со мной по вопросу доставки	5"""

df_pd28 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t28.split('\n')] )
 
df_pd28.columns = 'buy_id	buy_description	client_id'.split('\t')

mySchema = StructType([
                       T.StructField("buy_id", T.IntegerType(), True)\
                       , T.StructField("buy_description", T.StringType(), True)
                       , T.StructField("client_id", T.IntegerType(), True)
                       ])

df_sp28, df_pd28 = pDF_to_sDF( df_pd28
                          , table_name='buy'
                          , mySchema=mySchema
                          #, date_columns=['date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd28

,buy_id,buy_description,client_id
0,1,Доставка только вечером,1
1,2,,3
2,3,Упаковать каждую книгу по отдельности,2
3,4,,1
4,5,Связаться со мной по вопросу доставки,5


In [78]:
t29="""1	1	1	1
2	1	7	2
3	1	3	1
4	2	8	2
5	3	3	2
6	3	2	1
7	3	1	1
8	4	5	1
9	5	8	2
10	5	2	1"""

df_pd29 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t29.split('\n')] )
 
df_pd29.columns = 'buy_book_id	buy_id	book_id	amount'.split('\t')

mySchema = StructType([
                       T.StructField("buy_book_id", T.IntegerType(), True)\
                       , T.StructField("buy_id", T.IntegerType(), True)\
                       , T.StructField("book_id", T.IntegerType(), True)\
                       , T.StructField("amount", T.StringType(), True)
                       ])
df_sp29, df_pd29 = pDF_to_sDF( df_pd29
                          , table_name='buy_book'
                          , mySchema=mySchema
                          #, date_columns=['date_payment']
                          , int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd29

,buy_book_id,buy_id,book_id,amount
0,1,1,1,1
1,2,1,7,2
2,3,1,3,1
3,4,2,8,2
4,5,3,3,2
5,6,3,2,1
6,7,3,1,1
7,8,4,5,1
8,9,5,8,2
9,10,5,2,1


### Задание_1

In [79]:
# Full tables join

sql_250="""SELECT

    --buy
    bu.buy_id
    , bu.buy_description
    , bu.client_id --cc.client_id

    --client
    , cc.name_client --REVERSE(TO_BASE64(cc.name_client)) AS client64
    , cc.email --REVERSE(TO_BASE64(cc.email)) AS email
    , cc.city_id --ct.city_id

    --city
    , ct.name_city
    , ct.days_delivery
    
    --buy_step
    , bs.step_id --ss.step_id
    , bs.date_step_beg
    , bs.date_step_end
    
    --step
    , ss.name_step
    
    --buy_book
    , bb.book_id --bk.book_id
    , bb.amount
    
    --book
    , bk.title
    , bk.price
    , bk.amount
    
    --author
    , aa.name_author
    
    --genre
    , gg.name_genre

FROM buy AS bu

INNER JOIN client AS cc ON cc.client_id=bu.client_id
INNER JOIN city AS ct ON ct.city_id=cc.city_id

INNER JOIN buy_step AS bs ON bs.buy_id=bu.buy_id
INNER JOIN step AS ss ON ss.step_id=bs.step_id

INNER JOIN buy_book AS bb ON bb.buy_id=bu.buy_id
INNER JOIN book AS bk ON bb.book_id=bk.book_id

INNER JOIN author AS aa ON bk.author_id=aa.author_id
INNER JOIN genre AS gg ON bk.genre_id=gg.genre_id

ORDER BY bu.buy_id
        , bu.client_id
        , bs.step_id
        , bk.title
;"""

spark.sql(sql_250).toPandas()

,buy_id,buy_description,client_id,name_client,email,city_id,name_city,days_delivery,step_id,date_step_beg,date_step_end,name_step,book_id,amount,title,price,amount,name_author,name_genre
0,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,1,2020-02-20,2020-02-20,Оплата,3,1.0,Идиот,460.000000,10,Достоевский Ф.М.,Роман
1,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,1,2020-02-20,2020-02-20,Оплата,1,1.0,Мастер и Маргарита,670.989990,3,Булгаков М.А.,Роман
2,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,1,2020-02-20,2020-02-20,Оплата,7,2.0,Черный человек,570.200012,6,Есенин С.А.,Поэзия
3,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,2,2020-02-20,2020-02-21,Упаковка,3,1.0,Идиот,460.000000,10,Достоевский Ф.М.,Роман
4,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,2,2020-02-20,2020-02-21,Упаковка,1,1.0,Мастер и Маргарита,670.989990,3,Булгаков М.А.,Роман
5,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,2,2020-02-20,2020-02-21,Упаковка,7,2.0,Черный человек,570.200012,6,Есенин С.А.,Поэзия
6,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,3,2020-02-22,2020-03-07,Транспортировка,3,1.0,Идиот,460.000000,10,Достоевский Ф.М.,Роман
7,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,3,2020-02-22,2020-03-07,Транспортировка,1,1.0,Мастер и Маргарита,670.989990,3,Булгаков М.А.,Роман
8,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,3,2020-02-22,2020-03-07,Транспортировка,7,2.0,Черный человек,570.200012,6,Есенин С.А.,Поэзия
9,1,Доставка только вечером,1,Баранов Павел,baranov@test,3,Владивосток,12,4,2020-03-08,2020-03-08,Доставка,3,1.0,Идиот,460.000000,10,Достоевский Ф.М.,Роман


## 3.1 База данных «Тестирование», запросы на выборку

## Практика

In [80]:
t30 = """1	Основы SQL
2	Основы баз данных
3	Физика"""

df_pd30 = pd.DataFrame( [  [y.strip() for y in x.split("\t")] for x in t30.split('\n')] )
df_pd30.columns = 'subject_id	name_subject'.split('\t')

mySchema = StructType([
                       T.StructField("subject_id", T.IntegerType(), True)\
                       , T.StructField("name_subject", T.StringType(), True)
                       ])
df_sp30, df_pd30 = pDF_to_sDF( df_pd30
                          , table_name='subject'
                          , mySchema=mySchema
                          #, date_columns=['date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd30

,subject_id,name_subject
0,1,Основы SQL
1,2,Основы баз данных
2,3,Физика


In [81]:
t31 = """1	Баранов Павел
2	Абрамова Катя
3	Семенов Иван
4	Яковлева Галина"""

df_pd31 = pd.DataFrame( [ [y.strip() for y in x.split("\t")] for x in t31.split('\n')] )
df_pd31.columns = 'student_id	name_student'.split('\t')

df_sp31, df_pd31 = pDF_to_sDF( df_pd31
                          , table_name='student'
                          #, mySchema=mySchema
                          #, date_columns=['date_payment']
                          #, int_columns=['amount']
                          #, float_columns=['price']
                          )

df_pd31

,student_id,name_student
0,1,Баранов Павел
1,2,Абрамова Катя
2,3,Семенов Иван
3,4,Яковлева Галина


In [82]:
t32 = """1	1	2	2020-03-23	67
2	3	1	2020-03-23	100
3	4	2	2020-03-26	0
4	1	1	2020-04-15	33
5	3	1	2020-04-15	67
6	4	2	2020-04-21	100
7	3	1	2020-05-17	33"""

df_pd32 = pd.DataFrame( [ [y.strip() for y in x.split("\t")] for x in t32.split('\n')] )
df_pd32.columns = 'attempt_id	student_id	subject_id	date_attempt	result'.split('\t')

df_sp32, df_pd32 = pDF_to_sDF( df_pd32
                          , table_name='attempt'
                          #, mySchema=mySchema
                          , date_columns=['date_attempt']
                          , int_columns=['result']
                          #, float_columns=['price']
                          )

df_pd32

,attempt_id,student_id,subject_id,date_attempt,result
0,1,1,2,2020-03-23,67
1,2,3,1,2020-03-23,100
2,3,4,2,2020-03-26,0
3,4,1,1,2020-04-15,33
4,5,3,1,2020-04-15,67
5,6,4,2,2020-04-21,100
6,7,3,1,2020-05-17,33


In [83]:
t33 = """1	Запрос на выборку начинается с ключевого слова:	1
2	Условие, по которому отбираются записи, задается после ключевого слова:	1
3	Для сортировки используется:	1
4	Какой запрос выбирает все записи из таблицы student:	1
5	Для внутреннего соединения таблиц используется оператор:	1
6	База данных - это:	2
7	Отношение - это:	2
8	Концептуальная модель используется для	2
9	Какой тип данных не допустим в реляционной таблице?	2"""

df_pd33 = pd.DataFrame( [ [y.strip() for y in x.split("\t")] for x in t33.split('\n')] )
df_pd33.columns = 'question_id	name_question	subject_id'.split('\t')

df_sp33, df_pd33 = pDF_to_sDF( df_pd33
                          , table_name='question'
                          #, mySchema=mySchema
                          #, date_columns=['date_attempt']
                          #, int_columns=['result']
                          #, float_columns=['price']
                          )

df_pd33

,question_id,name_question,subject_id
0,1,Запрос на выборку начинается с ключевого слова:,1
1,2,"Условие, по которому отбираются записи, задает...",1
2,3,Для сортировки используется:,1
3,4,Какой запрос выбирает все записи из таблицы st...,1
4,5,Для внутреннего соединения таблиц используется...,1
5,6,База данных - это:,2
6,7,Отношение - это:,2
7,8,Концептуальная модель используется для,2
8,9,Какой тип данных не допустим в реляционной таб...,2


In [84]:
t34 = """1	UPDATE	1	false
2	SELECT	1	true
3	INSERT	1	false
4	GROUP BY	2	false
5	FROM	2	false
6	WHERE	2	true
7	SELECT	2	false
8	SORT 	3	false
9	ORDER BY	3	true
10	RANG BY	3	false
11	SELECT * FROM student	4	true
12	SELECT student	4	false
13	INNER JOIN	5	true
14	LEFT JOIN	5	false
15	RIGHT JOIN	5	false
16	CROSS JOIN	5	false
17	совокупность данных, организованных по определенным правилам	6	true
18	совокупность программ для хранения и обработки больших массивов информации	6	false
19	строка	7	false
20	столбец	7	false
21	таблица	7	true
22	обобщенное представление пользователей о данных 	8	true
23	описание представления данных в памяти компьютера 	8	false
24	база данных	8	false
25	file	9	true
26	INT	9	false
27	VARCHAR	9	false
28	DATE	9	false"""

df_pd34 = pd.DataFrame( [ [y.strip() for y in x.split("\t")] for x in t34.split('\n')] )
df_pd34.columns = 'answer_id	name_answer	question_id	is_correct'.split('\t')
df_pd34['is_correct'] = df_pd34['is_correct'].map(lambda x: True if x=='true' else False )

mySchema = StructType([
                       T.StructField("answer_id", T.IntegerType(), True) \
                       , T.StructField("name_answer", T.StringType(), True) \
                       , T.StructField("question_id", T.IntegerType(), True) \
                       , T.StructField("is_correct", T.BooleanType(), True)
                       ])
df_sp34, df_pd34 = pDF_to_sDF( df_pd34
                          , table_name='answer'
                          , mySchema=mySchema
                          #, date_columns=['date_attempt']
                          #, int_columns=['result']
                          #, float_columns=['price']
                          )

df_pd34

,answer_id,name_answer,question_id,is_correct
0,1,UPDATE,1,False
1,2,SELECT,1,True
2,3,INSERT,1,False
3,4,GROUP BY,2,False
4,5,FROM,2,False
5,6,WHERE,2,True
6,7,SELECT,2,False
7,8,SORT,3,False
8,9,ORDER BY,3,True
9,10,RANG BY,3,False


In [85]:
t35 = """1	1	9	25
2	1	7	19
3	1	6	17
4	2	3	9
5	2	1	2
6	2	4	11
7	3	6	18
8	3	8	24
9	3	9	28
10	4	1	2
11	4	5	16
12	4	3	10
13	5	2	6
14	5	1	2
15	5	4	12
16	6	6	17
17	6	8	22
18	6	7	21
19	7	1	3
20	7	4	11
21	7	5	16"""

df_pd35 = pd.DataFrame( [ [y.strip() for y in x.split("\t")] for x in t35.split('\n')] )
df_pd35.columns = 'testing_id	attempt_id	question_id	answer_id'.split('\t')

df_sp35, df_pd35 = pDF_to_sDF( df_pd35
                          , table_name='testing'
                          #, mySchema=mySchema
                          #, date_columns=['date_attempt']
                          #, int_columns=['result']
                          #, float_columns=['price']
                          )

df_pd35

,testing_id,attempt_id,question_id,answer_id
0,1,1,9,25
1,2,1,7,19
2,3,1,6,17
3,4,2,3,9
4,5,2,1,2
5,6,2,4,11
6,7,3,6,18
7,8,3,8,24
8,9,3,9,28
9,10,4,1,2


### Задание_1
<p>Вывести студентов, которые сдавали дисциплину «Основы баз данных», указать дату попытки и результат. Информацию вывести по убыванию результатов тестирования.</p>

<p><img alt="" src="https://ucarecdn.com/d46b4a8e-ef49-4335-815b-f3d685f63a13/">​</p>

In [86]:
sql_310="""SELECT

    ss.name_student
    , aa.date_attempt
    , aa.result
    
FROM attempt AS aa
INNER JOIN student AS ss ON aa.student_id=ss.student_id
INNER JOIN subject AS su ON aa.subject_id=su.subject_id AND su.name_subject='Основы баз данных'

ORDER BY aa.result DESC
;"""

spark.sql(sql_310).toPandas()

,name_student,date_attempt,result
0,Яковлева Галина,2020-04-21,100
1,Баранов Павел,2020-03-23,67
2,Яковлева Галина,2020-03-26,0


### Задание_2
<p>Вывести, сколько попыток сделали студенты по каждой дисциплине, а также средний результат попыток, который округлить до 2 знаков после запятой. Под результатом попытки понимается процент правильных ответов на вопросы теста, который занесен в столбец <code><strong>result</strong></code>.&nbsp; В результат включить название дисциплины, а также вычисляемые столбцы <code><strong>Количество</strong></code> и <code><strong>Среднее</strong></code>. Информацию вывести по убыванию средних результатов.</p>

<p><img alt="" src="https://ucarecdn.com/d8715483-b850-4ce8-8e2d-12ac8ba18d0c/"></p>

In [87]:
sql_311="""SELECT

    su.name_subject

    , COUNT(aa.date_attempt) AS Kolichestvo
    , ROUND(AVG(aa.result),2) AS Srednee

FROM subject AS su
LEFT JOIN attempt AS aa ON aa.subject_id=su.subject_id 

GROUP BY su.name_subject

ORDER BY Srednee DESC
;"""

spark.sql(sql_311).toPandas()

,name_subject,Kolichestvo,Srednee
0,Основы SQL,4,58.25
1,Основы баз данных,3,55.67
2,Физика,0,NaN


### Задание_3
Вывести студентов (различных студентов), имеющих максимальные результаты попыток. Информацию отсортировать в алфавитном порядке по фамилии студента.

Максимальный результат не обязательно будет 100%, поэтому явно это значение в запросе не задавать.

<p><img alt="" src="https://ucarecdn.com/3d7b1b34-90b4-4d76-aea0-03369872489d/"></p>

In [88]:
sql_312="""SELECT

    --su.name_subject
    ss.name_student
    , aa.result

    --, COUNT(aa.date_attempt) AS Kolichestvo
    --, ROUND(AVG(aa.result),2) AS Srednee

FROM subject AS su
LEFT JOIN attempt AS aa ON aa.subject_id=su.subject_id 
INNER JOIN student AS ss ON aa.student_id=ss.student_id
--AND su.name_subject='Основы баз данных'

WHERE aa.result = (SELECT MAX(result) FROM attempt)

ORDER BY ss.name_student
;"""

spark.sql(sql_311).toPandas()

,name_subject,Kolichestvo,Srednee
0,Основы SQL,4,58.25
1,Основы баз данных,3,55.67
2,Физика,0,NaN


### Задание_4
<p>Если студент совершал <strong>несколько попыток</strong> по одной и той же дисциплине, то вывести разницу в днях между первой и последней попыткой. В результат включить фамилию и имя студента, название дисциплины и вычисляемый столбец <code><strong>Интервал</strong></code>. Информацию вывести по возрастанию разницы. Студентов, сделавших одну попытку по дисциплине, не учитывать.&nbsp;</p>

<p><img alt="" src="https://ucarecdn.com/0702acbe-ec20-457d-85e4-6df22e070656/"></p>

In [89]:
sprint("""SELECT

    ss.name_student
    , su.name_subject
    , DATEDIFF(MAX(aa.date_attempt),MIN(aa.date_attempt)) AS Интервал

FROM subject AS su
LEFT JOIN attempt AS aa ON aa.subject_id=su.subject_id 
INNER JOIN student AS ss ON aa.student_id=ss.student_id

GROUP BY ss.name_student
    , su.name_subject

HAVING MIN(aa.date_attempt)<>MAX(aa.date_attempt)
    
ORDER BY Интервал
;""")

### Задание_5
<p>Студенты могут тестироваться&nbsp;по одной или нескольким дисциплинам (не обязательно по всем). Вывести дисциплину и количество уникальных студентов (столбец назвать <code><strong>Количество</strong></code>), которые по ней&nbsp;проходили тестирование . Информацию отсортировать сначала&nbsp;по убыванию количества, а потом по названию дисциплины. В результат включить и дисциплины, тестирование по которым студенты еще не проходили, в этом случае указать количество студентов 0.</p>

In [90]:
sprint("""SELECT

    su.name_subject
    , COUNT(DISTINCT(ss.name_student)) AS Количество
    
FROM subject AS su
LEFT JOIN attempt AS aa ON aa.subject_id=su.subject_id 
LEFT JOIN student AS ss ON aa.student_id=ss.student_id

GROUP BY su.name_subject

ORDER BY Количество DESC
        , su.name_subject
;""")

### Задание_6
<p>Случайным образом отберите 3 вопроса по дисциплине «Основы баз данных». В результат включите столбцы <code><strong>question_id</strong></code> и <code><strong>name_question</strong></code>.</p>

<p><img alt="" src="https://ucarecdn.com/332c38fe-08e4-4c27-bc71-3b3200f43c3b/"></p>

<details open=""><summary><strong>Пояснение</strong></summary>

<p>Для выбора случайных вопросов можно отсортировать вопросы в случайном порядке:</p>

<pre><code class="language-sql hljs">ORDER BY RAND()</code></pre>
</details>

In [91]:
sprint("""SELECT

    qq.question_id
    , qq.name_question
    
FROM subject AS su
INNER JOIN question as qq ON qq.subject_id=su.subject_id

WHERE su.name_subject = 'Основы баз данных'

ORDER BY RAND()

LIMIT 3
;""")

### Задание_7

<p>Вывести вопросы, которые были включены в тест для Семенова Ивана по дисциплине «Основы SQL»&nbsp;2020-05-17&nbsp; (значение <code><strong>attempt_id</strong></code><strong> </strong>для этой попытки равно&nbsp;7). Указать, какой ответ дал студент и правильный он или нет (вывести <strong>Верно</strong> или <strong>Неверно</strong>). В результат включить вопрос, ответ и вычисляемый столбец &nbsp;<code><strong>Результат</strong></code>.</p>

<p><img alt="" src="https://ucarecdn.com/9cea1edc-98ee-4215-ae29-70ebf8aaea21/"></p>

In [92]:
sprint("""SELECT

    qq.name_question
    , an.name_answer
    , IF(an.is_correct=1,'Верно','Неверно') AS Результат
    
FROM testing AS tt
LEFT JOIN question as qq ON qq.question_id=tt.question_id
LEFT JOIN answer as an ON an.answer_id=tt.answer_id

WHERE tt.attempt_id = 7
;""")

### Задание_8
<p>Посчитать результаты&nbsp;тестирования. Результат попытки вычислить как количество правильных ответов, деленное на 3 (количество вопросов в каждой попытке) и умноженное на 100. Результат округлить до двух знаков после запятой. Вывести фамилию студента, название предмета, дату и результат. Последний столбец назвать <code><strong>Результат</strong></code>. Информацию отсортировать сначала по фамилии студента, потом по убыванию даты попытки.</p>

<p><img alt="" src="https://ucarecdn.com/e4669333-8898-434f-b1a5-4fa88b39ae02/"></p>

In [93]:
sprint("""SELECT

    ss.name_student
    , xx.name_subject
    , aa.date_attempt
    
    --, COUNT( tt.answer_id ) AS cnt
    --, SUM( xx.is_correct ) AS tru
    
    , ROUND((SUM(
        xx.is_correct )/COUNT( tt.answer_id )*100
            ),2) AS Результат

FROM attempt AS aa 
LEFT JOIN student AS ss ON ss.student_id=aa.student_id
LEFT JOIN testing AS tt ON aa.attempt_id=tt.attempt_id 

LEFT JOIN ( SELECT

                uu.subject_id
                , uu.name_subject
                , ww.question_id    
                , ww.answer_id
                , ww.is_correct

            FROM subject AS uu
            LEFT JOIN question AS qq ON uu.subject_id=qq.subject_id
            LEFT JOIN answer AS ww ON qq.question_id=ww.question_id
) AS xx
    ON tt.question_id=xx.question_id
    AND tt.answer_id=xx.answer_id
    AND aa.subject_id=xx.subject_id

--WHERE ss.name_student='Баранов Павел'

GROUP BY     
    ss.name_student
    , xx.name_subject
    , aa.date_attempt

ORDER BY ss.name_student ASC
    , aa.date_attempt DESC
;""")

### Задание_9
<p>Для каждого вопроса вывести процент успешных решений, то есть отношение количества верных ответов к общему количеству ответов, значение округлить до 2-х знаков после запятой. Также вывести название предмета, к которому относится вопрос,&nbsp;и общее количество ответов на этот вопрос. В результат включить название дисциплины, вопросы по ней (столбец назвать <code><strong>Вопрос</strong></code>), а также два вычисляемых столбца <code><strong>Всего_ответов</strong></code> и <code><strong>Успешность</strong></code>. Информацию отсортировать сначала по названию дисциплины, потом по убыванию успешности, а потом по тексту вопроса в алфавитном порядке.</p>

<p>Поскольку тексты вопросов&nbsp;могут быть длинными, обрезать их 30 символов и добавить многоточие "...".</p>

In [94]:
sprint("""SELECT
    xx.name_subject 
    , CONCAT(
         LEFT(xx.name_question,30)
        , '...') AS Вопрос
    
    -- ss.name_student
    -- aa.subject_id
    -- aa.date_attempt
    -- aa.attempt_id
    -- tt.testing_id
    -- tt.question_id
    -- tt.answer_id
    
    --, SUM( xx.is_correct ) AS tru
    
    , COUNT( xx.is_correct ) AS Всего_ответов
    
    , ROUND((SUM(
        xx.is_correct )/COUNT( xx.is_correct )*100
            ),2) AS Успешность

FROM attempt AS aa 
LEFT JOIN student AS ss ON ss.student_id=aa.student_id
LEFT JOIN testing AS tt ON aa.attempt_id=tt.attempt_id 

LEFT JOIN ( SELECT

                uu.subject_id
                , uu.name_subject
                , ww.question_id    
                , qq.name_question
                , ww.answer_id
                , ww.is_correct

            FROM subject AS uu
            LEFT JOIN question AS qq ON uu.subject_id=qq.subject_id
            LEFT JOIN answer AS ww ON qq.question_id=ww.question_id
) AS xx
    ON tt.question_id=xx.question_id
    AND tt.answer_id=xx.answer_id
    AND aa.subject_id=xx.subject_id

-- WHERE ss.name_student='Баранов Павел'

GROUP BY     
    xx.name_subject
    , Вопрос

ORDER BY xx.name_subject ASC
    , Успешность DESC
    , Вопрос ASC
;""")